In [ ]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge                 # 모델 성능평가 라이브러리

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

from pathlib import Path
from transformers import BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datetime import datetime
from difflib import SequenceMatcher
from collections import Counter
from itertools import tee, islice

import wandb
wandb.login(key="")                     # API KEY 입력

# wandb 꺼두기
os.environ["WANDB_DISABLED"] = "true"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


### Config 정의

In [51]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")

In [52]:
config_data = {
    "general": {
        "data_path": "../data/",
        "data_raw_path": "../data/raw/",
        "data_processed_path": "../data/processed/",                                       
        "model_name": "digit82/kobart-summarization",                           # 모델 이름
        "output_dir": "../outputs/checkpoints/"                                 
    },
    "tokenizer": {
        "encoder_max_len": 512,
        "decoder_max_len": 100,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#',
                           '#PhoneNumber#', '#Address#', '#PassportNumber#']    # 특정 단어들이 분해되지 않도록 하기 위함
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 50,
        "per_device_eval_batch_size": 32,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 100,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "none"  
        # "report_to": "wandb"                                                        # wandb 사용 옵션
    },
    "wandb": {
        # "entity": "bubblekid43",                                                   
        "project": "NLP-private",                                                   
        "name": "preprocessed_v1"                                                       # 버전관리
    },
    "inference": {
        "ckt_path": "../outputs/checkpoints/",                                      
        "result_path": "../outputs/prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 100,
        "num_beams": 4,
        "batch_size" : 32,
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}",
                          f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]       # 불필요한 생성 토큰 정의
    },
}

In [53]:
# 모델의 구성 정보를 YAML 파일로 저장
config_path = "../configs/config_v1.yaml"                                            # 버전관리
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)
print(f"💾 config 저장 완료 : {config_path}")

# 저장된 config 파일 불러와서 내용 확인
with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)
print("\n🔍 config 구성 정보 확인")
print(loaded_config)

💾 config 저장 완료 : ../configs/config_v1.yaml

🔍 config 구성 정보 확인
{'general': {'data_path': '../data/', 'data_processed_path': '../data/processed/', 'data_raw_path': '../data/raw/', 'model_name': 'digit82/kobart-summarization', 'output_dir': '../outputs/checkpoints/'}, 'inference': {'batch_size': 32, 'ckt_path': '../outputs/checkpoints/', 'early_stopping': True, 'generate_max_length': 100, 'no_repeat_ngram_size': 2, 'num_beams': 4, 'remove_tokens': ['<usr>', '<s>', '</s>', '<pad>'], 'result_path': '../outputs/prediction/'}, 'tokenizer': {'bos_token': '<s>', 'decoder_max_len': 100, 'encoder_max_len': 512, 'eos_token': '</s>', 'special_tokens': ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']}, 'training': {'do_eval': True, 'do_train': True, 'early_stopping_patience': 3, 'early_stopping_threshold': 0.001, 'evaluation_strategy': 'epoch', 'fp16': True, 'generation_max_length': 100, 'gradient_accumulation_steps': 1, 'learning_rate': 1e-05, 'load_best_mod

### 데이터 로드

In [54]:
# 데이터 경로 정의
data_path = Path(loaded_config['general']['data_path'])
data_processed_path = Path(loaded_config['general']['data_processed_path'])

In [55]:
# train data
train_df = pd.read_csv(data_processed_path /'preprocessed_v1' / 'train_preprocessed_v1.csv')     # 버전관리
print('✅ train_df :', train_df.shape)
display(train_df.head())

✅ train_df : (12457, 4)


,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...","Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",건강검진
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...","Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",백신 접종
2,train_2,"#Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 떤 종...",#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...,열쇠 분실
3,train_3,#Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...,여자친구와의 결혼
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...","Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",춤 제안


In [56]:
# validation data
val_df = pd.read_csv(data_processed_path / 'preprocessed_v1' / 'dev_preprocessed_v1.csv')     # 버전관리
print('✅ val_df :', val_df.shape)
display(val_df.head())

✅ val_df : (499, 4)


,fname,dialogue,summary,topic
0,dev_0,"#Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...",#Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...,의사 상담
1,dev_1,"#Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...",#Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.,운동 계획
2,dev_2,#Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어.\n#Pers...,"#Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...",건강한 식단
3,dev_3,"#Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...",#Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...,UFO와 외계인
4,dev_4,#Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...,"#Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...",학교와 주말 계획


In [57]:
# test data
test_df = pd.read_csv(data_processed_path / 'preprocessed_v1' / 'test_preprocessed_v1.csv')   # 버전관리
print('✅ test_df:', test_df.shape)
display(test_df.head())

✅ test_df: (499, 2)


,fname,dialogue
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요.\n#Pers..."
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person..."
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per..."
4,test_4,#Person1#: 이 올림픽 공원 정말 크다!\n#Person2#: 맞아. 지금 ...


### EDA 설정

In [58]:
#========== dialogue ==========#

# 각 줄의 좌우 공백 찾는 함수
def has_leading_or_trailing_spaces(text: str) -> bool:
    return any(
        (line.startswith(" ") or line.endswith(" "))
        for line in text.split("\n")
    )

# 연속 공백(2칸 이상) 찾는 함수
def has_multi_spaces(text: str) -> bool:
    return bool(re.search(r"[ \t]{2,}", text))

#========== summary ==========#

# Summary 문장 수 (한 문장 위주인지 확인) -> 거의 한 문장이라고 봐야할 듯
def count_sentences(text: str) -> int:
    if not isinstance(text, str):
        return 0
    # 마침표/물음표/느낌표 기준으로 대략 문장 수 세기
    parts = re.split(r"[.!?]\s*", text.strip())
    # 빈 문자열 제거
    parts = [p for p in parts if p]
    return len(parts)

# Summary가 대화의 “어느 위치” 정보를 반영하는지 분석
def split_turns(dialogue):
    return [t.strip() for t in dialogue.split("\n") if t.strip()] # 발화 단위로 분리

# Summary와 가장 유사한 turn이 처음, 중간, 끝 중 어디에 위치하는지
def find_related_turn(dialogue, summary):
    turns = split_turns(dialogue)
    sims = []
    for i, turn in enumerate(turns):
        ratio = SequenceMatcher(None, turn, summary).ratio()
        sims.append((i, ratio))
    best = max(sims, key=lambda x: x[1])
    return best  # (turn_index, similarity_score)

# Summary에 남는 / 삭제되는 키워드 확인하는 함수
def simple_tokenize(text: str):
    text = text.lower()
    text = re.sub(r"[^가-힣A-Za-z0-9 ]", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    if not text:
        return []
    tokens = text.split(" ")
    return tokens

def keyword_mapping(dialogue, summary):
    d_words = set(simple_tokenize(dialogue))
    s_words = set(simple_tokenize(summary))

    kept = s_words & d_words
    dropped = d_words - s_words
    
    return list(kept), list(dropped)

# Summary 종결 표현 스타일
def get_ending(text: str, n_chars: int = 3) -> str:
    if not isinstance(text, str) or not text.strip():
        return ""
    t = text.strip()
    # 너무 짧으면 전체 반환
    if len(t) <= n_chars:
        return t
    return t[-n_chars:]

#========== topic ==========#

# Topic과 Summary 관계
def topic_in_summary(row):
    topic = str(row["topic"])
    summ = str(row["summary"])
    return topic in summ

# topic별로 패턴 대략 보는 코드
def topic_coverage_ratio(row):
    topic_tokens = simple_tokenize(str(row["topic"]))
    summary_tokens = simple_tokenize(str(row["summary"]))

    if not topic_tokens:
        return 0.0

    covered = 0
    for tt in topic_tokens:
        # 부분 일치 또는 완전 일치 모두 인정
        if any((tt in st) or (st in tt) for st in summary_tokens):
            covered += 1

    return covered / len(topic_tokens)

# 유니그램/바이그램 overlap 계산 함수
def make_bigrams(tokens: list[str]) -> list[tuple[str, str]]:
    if len(tokens) < 2:
        return []
    t1, t2 = tee(tokens)
    next(t2, None)
    return list(zip(t1, t2))

def lexical_overlap(dialogue: str, summary: str) -> dict:
    d_tokens = simple_tokenize(dialogue)
    s_tokens = simple_tokenize(summary)
    
    d_set = set(d_tokens)
    s_set = set(s_tokens)
    
    # 유니그램 overlap (요약 토큰 기준 커버리지)
    if len(s_set) > 0:
        uni_overlap_ratio = len(d_set & s_set) / len(s_set)
    else:
        uni_overlap_ratio = 0.0

    # bigram overlap
    d_bi = set(make_bigrams(d_tokens))
    s_bi = set(make_bigrams(s_tokens))
    if len(s_bi) > 0:
        bi_overlap_ratio = len(d_bi & s_bi) / len(s_bi)
    else:
        bi_overlap_ratio = 0.0
    
    return {
        "uni_overlap": uni_overlap_ratio,
        "bi_overlap": bi_overlap_ratio,
        "summary_uniq_tokens": len(s_set),
        "dialogue_uniq_tokens": len(d_set),
    }

In [59]:
#========== dialogue ==========#
# dialogue 길이
train_df["dialogue_len"] = train_df["dialogue"].str.len()
val_df["dialogue_len"] = val_df["dialogue"].str.len()
test_df["dialogue_len"] = test_df["dialogue"].str.len()
train_df["dialogue_len_token"] = train_df["dialogue"].str.split().str.len()
val_df["dialogue_len_token"] = val_df["dialogue"].str.split().str.len()
test_df["dialogue_len_token"] = test_df["dialogue"].str.split().str.len()

# dialogue 공백
train_df["d_has_space_edges"] = train_df["dialogue"].apply(has_leading_or_trailing_spaces)
val_df["d_has_space_edges"] = val_df["dialogue"].apply(has_leading_or_trailing_spaces)
test_df["d_has_space_edges"] = test_df["dialogue"].apply(has_leading_or_trailing_spaces)

train_df["d_has_multi_spaces"] = train_df["dialogue"].apply(has_multi_spaces)
val_df["d_has_multi_spaces"] = val_df["dialogue"].apply(has_multi_spaces)
test_df["d_has_multi_spaces"] = test_df["dialogue"].apply(has_multi_spaces)

#========== summary ==========#
# summary 길이
train_df["summary_n_sent"] = train_df["summary"].apply(count_sentences)
val_df["summary_n_sent"] = val_df["summary"].apply(count_sentences)
train_df["summary_len"] = train_df["summary"].str.len()
val_df["summary_len"] = val_df["summary"].str.len()

# dialogue 대비 summary 길이 (요약 압축률)
train_df["len_ratio"] = train_df["summary_len"] / train_df["dialogue_len"]
val_df["len_ratio"] = val_df["summary_len"] / val_df["dialogue_len"]

# dialogue ↔ summary lexical overlap 분석
lexical_df = train_df.apply(lambda row: lexical_overlap(row["dialogue"], row["summary"]), axis=1)
lexical_df = pd.DataFrame(list(lexical_df))
train_df = pd.concat([train_df, lexical_df], axis=1)
lexical_df = val_df.apply(lambda row: lexical_overlap(row["dialogue"], row["summary"]), axis=1)
lexical_df = pd.DataFrame(list(lexical_df))
val_df = pd.concat([val_df, lexical_df], axis=1)

# summary 공백
train_df["s_has_space_edges"] = train_df["summary"].apply(has_leading_or_trailing_spaces)
val_df["s_has_space_edges"] = val_df["summary"].apply(has_leading_or_trailing_spaces)

train_df["s_has_multi_spaces"] = train_df["summary"].apply(has_multi_spaces)
val_df["s_has_multi_spaces"] = val_df["summary"].apply(has_multi_spaces)

train_df["s_has_p1"] = train_df["summary"].str.contains("#Person1#")
train_df["s_has_p2"] = train_df["summary"].str.contains("#Person2#")
val_df["s_has_p1"] = val_df["summary"].str.contains("#Person1#")
val_df["s_has_p2"] = val_df["summary"].str.contains("#Person2#")

# summary와 제일 유사한 dialogue 줄
train_df["summary_main_turn"] = train_df.apply(lambda r: find_related_turn(r["dialogue"], r["summary"])[0], axis=1)
val_df["summary_main_turn"] = val_df.apply(lambda r: find_related_turn(r["dialogue"], r["summary"])[0], axis=1)

# summary에 남는 or 삭제되는 키워드
train_df["keywords_kept"], train_df["keywords_dropped"] = zip(*train_df.apply(lambda r: keyword_mapping(r["dialogue"], r["summary"]), axis=1))
val_df["keywords_kept"], val_df["keywords_dropped"] = zip(*val_df.apply(lambda r: keyword_mapping(r["dialogue"], r["summary"]), axis=1))

# summary 종결 표현
train_df["summary_ending3"] = train_df["summary"].apply(lambda x: get_ending(x, 3))
train_df["summary_ending2"] = train_df["summary"].apply(lambda x: get_ending(x, 2))
val_df["summary_ending3"] = val_df["summary"].apply(lambda x: get_ending(x, 3))
val_df["summary_ending2"] = val_df["summary"].apply(lambda x: get_ending(x, 2))

#========== topic ==========#
train_df["topic_stripped"] = train_df["topic"].astype(str).str.strip()      # 앞뒤 공백 제거한 버전
val_df["topic_stripped"] = val_df["topic"].astype(str).str.strip()
train_df["topic_len"] = train_df["topic_stripped"].str.len()                # topic 길이
val_df["topic_len"] = val_df["topic_stripped"].str.len()

# Topic이 summary 안에 직접 등장하는지 여부
train_df["topic_in_summary"] = train_df.apply(topic_in_summary, axis=1)
val_df["topic_in_summary"] = val_df.apply(topic_in_summary, axis=1)

# topic token이 summary에서 몇 % 커버되는지
train_df["topic_coverage"] = train_df.apply(topic_coverage_ratio, axis=1)
val_df["topic_coverage"] = val_df.apply(topic_coverage_ratio, axis=1)

In [60]:
#========== 컬럼 순서 지정 ==========#
# train, val
cols_train=[
    "fname",
    "dialogue",
    "dialogue_len",
    "dialogue_len_token",
    "d_has_space_edges",
    "d_has_multi_spaces",
    "summary",
    "summary_n_sent",
    "summary_len",
    "len_ratio",
    "uni_overlap", "bi_overlap",
    "s_has_space_edges",
    "s_has_multi_spaces",
    "s_has_p1",
    "s_has_p2",
    "summary_main_turn",
    "keywords_kept",
    "keywords_dropped",
    "summary_ending3",
    "summary_ending2",
    "topic",
    "topic_stripped",
    "topic_len",
    "topic_in_summary",
    "topic_coverage",
]

# test
cols_test=[
    "fname",
    "dialogue",
    "dialogue_len",
    "dialogue_len_token",
    "d_has_space_edges",
    "d_has_multi_spaces",
]

In [61]:
# 데이터 살펴보기
print('✅ train_df :', train_df[cols_train].shape)
display(train_df[cols_train].head())

print('✅ val_df :', val_df[cols_train].shape)
display(val_df[cols_train].head())

print('✅ test_df:', test_df[cols_test].shape)
display(test_df[cols_test].head())

✅ train_df : (12457, 26)


,fname,dialogue,dialogue_len,dialogue_len_token,d_has_space_edges,d_has_multi_spaces,summary,summary_n_sent,summary_len,len_ratio,...,summary_main_turn,keywords_kept,keywords_dropped,summary_ending3,summary_ending2,topic,topic_stripped,topic_len,topic_in_summary,topic_coverage
0,train_0,"#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다...",518,105,False,False,"Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필...",3,80,0.154440,...,0,"[mr, 건강검진을, 매년, dr]","[해요, 동안, person1, 번은, 시도했는데, 없으면, 쉬어보세요, 끊으셔야,...",니다.,다.,건강검진,건강검진,4,True,1.0
1,train_1,"#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?\n#Pers...",433,92,False,False,"Mrs. Parker가 Ricky와 함께 백신 접종을 위해 방문하였고, Dr. Pe...",3,80,0.184758,...,1,"[백신, mrs, dr]","[홍역, 해요, 수, person1, ricky는, 지금은, 예방접종을, 소아마비,...",니다.,다.,백신 접종,백신 접종,5,True,1.0
2,train_2,"#Person1#: 저기요, 열쇠 세트 본 적 있어요?\n#Person2#: 떤 종...",289,52,False,False,#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...,1,54,0.186851,...,0,"[person2, person1, 열쇠]","[수, 안타깝네요, 장식이, 다섯, 찾아주실, 같이, 이봐요, 종류의, 친절하시네요...",니다.,다.,열쇠 분실,열쇠 분실,5,False,0.5
3,train_3,#Person1#: 너 여자친구 있는 거 왜 말 안 했어?\n#Person2#: 미...,311,63,False,False,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...,1,61,0.196141,...,0,"[person2, 결혼할, person1, 가]","[중요하지, 수도, 했잖아, 사람, 말하잖아, 내가, 왜, 남자들, 그런데, 미안해...",니다.,다.,여자친구와의 결혼,여자친구와의 결혼,9,False,0.5
4,train_4,"#Person1#: 안녕, 오늘 너무 멋져 보이네요. 저랑 춤 한 곡 추실래요?\n...",277,55,False,False,"Malik은 Wen과 Nikki에게 춤을 제안하고, Wen은 발을 밟는 것을 감수하...",1,59,0.212996,...,5,[발을],"[춤, 반가워요, 가요, 추실래요, 제가, person1, 친구랑, 잘, 그런데, ...",니다.,다.,춤 제안,춤 제안,4,False,1.0


✅ val_df : (499, 26)


,fname,dialogue,dialogue_len,dialogue_len_token,d_has_space_edges,d_has_multi_spaces,summary,summary_n_sent,summary_len,len_ratio,...,summary_main_turn,keywords_kept,keywords_dropped,summary_ending3,summary_ending2,topic,topic_stripped,topic_len,topic_in_summary,topic_coverage
0,dev_0,"#Person1#: 안녕하세요, 오늘 기분이 어떠세요?\n#Person2#: 요즘 ...",333,64,False,False,#Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...,2,78,0.234234,...,8,"[폐, 천식, 위해, person2, 검사를, 전문의에게]","[힘들어요, 많이, 그런가요, 있으세요, person1, 도와주셔서, 이게, 니요,...",니다.,다.,의사 상담,의사 상담,5,False,0.500000
1,dev_1,"#Person1#: 야 Jimmy, 오늘 좀 이따 운동하러 가자.\n#Person2...",280,60,False,False,#Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.,1,48,0.171429,...,0,"[복근, person1, 운동하러]","[일정이, 두, 어때, 아까, 보자, 3시, jimmy, 날이야, 다리가, 다, 시...",니다.,다.,운동 계획,운동 계획,5,False,0.500000
2,dev_2,#Person1#: 나 진짜 건강에 안 좋은 음식 좀 그만 먹어야겠어.\n#Pers...,308,63,False,False,"#Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#...",1,78,0.253247,...,0,"[건강에, person2, person1, 그만, 좋은, 안]","[닭고기는, 말인지, 채소, 하거든, 보이네, 나도, 왜, 음식, 구워, 먹으려고,...",니다.,다.,건강한 식단,건강한 식단,6,True,1.000000
3,dev_3,"#Person1#: 너 UFO 믿어?\n#Person2#: 당연하지, 있는 거 아냐...",430,89,False,False,#Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...,2,107,0.248837,...,7,"[수, 볼, person2, person1, 꿈에서]","[얘기해, 아니야, 생겼어, 화성에서, 외계인, 자기네, 언어로, 이고, 그게, 임...",한다.,다.,UFO와 외계인,UFO와 외계인,8,True,1.000000
4,dev_4,#Person1#: 오늘 학교 갔어?\n#Person2#: 당연히 갔지. 너는?\n...,315,67,False,False,"#Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ...",1,60,0.190476,...,7,"[오늘, 가고, person2, person1, 학교]","[싫어, 갈, 지는, 왜, 갈까, 갔어, 갔지, 진지하게, 무튼, 중이야, 근데, ...",니다.,다.,학교와 주말 계획,학교와 주말 계획,9,False,0.333333


✅ test_df: (499, 6)


,fname,dialogue,dialogue_len,dialogue_len_token,d_has_space_edges,d_has_multi_spaces
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요.\n#Pers...",687,141,False,False
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...,559,123,False,False
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person...",436,89,False,False
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per...",366,78,False,False
4,test_4,#Person1#: 이 올림픽 공원 정말 크다!\n#Person2#: 맞아. 지금 ...,343,69,False,False


In [62]:
# dialogue noise 시각적 EDA
    # 정보 요약 중심 스타일
    # 감정·감탄·상대적 표현·반응성 문장 > 공격적 제거 대상
    # 상황, 주요 행동/사실, 결과/다음 조치 > 중심 요약 대상

# print("val_df.shape: ", val_df.shape, "\n")

# n=150
# print("✅ dialogue")
# print(val_df.iloc[n]["dialogue"], "\n")

# print("✅ summary")
# print(val_df.iloc[n]["summary"])

### Dialogue

In [63]:
# 각 dialogue에서 등장하는 화자 태그 전체 추출
def extract_speaker_tags(text):
    pattern = r"#Person\d+[^A-Za-z0-9가-힣]?"       # 예: #Person1#: / #Person1# : / #Person1 :
    return re.findall(pattern, text)

# train.csv
all_tags_train = []
for s in train_df["dialogue"]:
    all_tags_train.extend(extract_speaker_tags(s))

# dev.csv
all_tags_val = []
for t in val_df["dialogue"]:
    all_tags_val.extend(extract_speaker_tags(t))

# test.csv
all_tags_test = []
for u in test_df["dialogue"]:
    all_tags_test.extend(extract_speaker_tags(u))


#=============================================================================#
print("📊 화자 태그 패턴 분포")
print("➡️ train_df")
tag_counts_train = pd.Series(all_tags_train).value_counts()
print(tag_counts_train)
print("➡️ val_df")
tag_counts_val = pd.Series(all_tags_val).value_counts()
print(tag_counts_val)
print("➡️ test_df")
tag_counts_test = pd.Series(all_tags_test).value_counts()
print(tag_counts_test)

#=============================================================================#
print("\n🔎 화자 태그 중복 여부 확인")
print("➡️ train_df :", pd.Series(tag_counts_train.index).to_list())
print("➡️ val_df   :", pd.Series(tag_counts_val.index).to_list())
print("➡️ test_df  :", pd.Series(tag_counts_test.index).to_list())

📊 화자 태그 패턴 분포
➡️ train_df
#Person1#    61361
#Person2#    56866
#Person3#        7
#Person4#        1
Name: count, dtype: int64
➡️ val_df
#Person1#    2439
#Person2#    2251
Name: count, dtype: int64
➡️ test_df
#Person1#    2518
#Person2#    2321
Name: count, dtype: int64

🔎 화자 태그 중복 여부 확인
➡️ train_df : ['#Person1#', '#Person2#', '#Person3#', '#Person4#']
➡️ val_df   : ['#Person1#', '#Person2#']
➡️ test_df  : ['#Person1#', '#Person2#']


In [64]:
#========== dialogue 당 줄바꿈 개수 ==========#
newline_counts_train = train_df["dialogue"].str.count("\n")
newline_counts_val = val_df["dialogue"].str.count("\n")
newline_counts_test = test_df["dialogue"].str.count("\n")

print("\n📊 줄바꿈 개수 분포")
print("➡️ train_df")
display(newline_counts_train.describe())
print("➡️ val_df")
display(newline_counts_val.describe())
print("➡️ test_df")
display(newline_counts_test.describe())

print("\n🔎 줄바꿈 개수 샘플 (상위 10개)")
print("➡️ train_df")
display(train_df.assign(n_newlines=newline_counts_train).sort_values("n_newlines", ascending=False).head()[["fname", "n_newlines", "dialogue"]])
print("➡️ val_df")
display(val_df.assign(n_newlines=newline_counts_val).sort_values("n_newlines", ascending=False).head()[["fname", "n_newlines", "dialogue"]])
print("➡️ test_df")
display(test_df.assign(n_newlines=newline_counts_test).sort_values("n_newlines", ascending=False).head()[["fname", "n_newlines", "dialogue"]])


#========== 연속 줄바꿈이 있는 행 확인 ==========#
multi_nl_mask_train = train_df["dialogue"].str.contains(r"\n{2,}", regex=True)
multi_nl_mask_val = val_df["dialogue"].str.contains(r"\n{2,}", regex=True)
multi_nl_mask_test = train_df["dialogue"].str.contains(r"\n{2,}", regex=True)

print("⚠️ 연속 줄바꿈(\\n\\n) 포함된 행의 개수")
print("➡️ train_df :", multi_nl_mask_train.sum())
print("➡️ val_df   :", multi_nl_mask_val.sum())
print("➡️ test_df  :", multi_nl_mask_test.sum())


📊 줄바꿈 개수 분포
➡️ train_df


count    12457.000000
mean         8.478526
std          4.150184
min          0.000000
25%          6.000000
50%          8.000000
75%         11.000000
max         58.000000
Name: dialogue, dtype: float64

➡️ val_df


count    499.000000
mean       8.388778
std        4.017676
min        1.000000
25%        5.000000
50%        8.000000
75%       11.000000
max       28.000000
Name: dialogue, dtype: float64

➡️ test_df


...

➡️ val_df


,fname,n_newlines,dialogue
334,dev_334,28,"#Person1#: 저기, 장 본 것 좀 도와줄래? 내가 쇼핑해도 된다고 했잖아.\..."
183,dev_183,24,#Person1#: 너 컴퓨터 키보드 다 달아버릴 거야!\n#Person2#: 안녕...
410,dev_410,22,#Person1#: 내일 밤에 뭐 하고 싶어?\n#Person2#: 뭐라고? 내일 ...
441,dev_441,20,#Person1#: 해가 떴어! 모든 캠핑객들 일어나야지. 노 젓기 시작할 시간이야...
338,dev_338,19,#Person1#: 대학 갔어?\n#Person2#: 칭다오 대학에서 영문학 학사로...


➡️ test_df


,fname,n_newlines,dialogue
434,test_434,62,"#Person1#: 안녕, Andrew! 안녕 ...? Andrew.\n#Perso..."
87,test_87,36,"#Person1#: 좋은 아침이에요, Ms. Ross. 들어오세요.\n#Person..."
483,test_484,34,#Person1#: 저기요.\n#Person2#: ?\n#Person1#: 여기 처...
184,test_184,27,#Person1#: 안녕하세요. 도와드릴까요?\n#Person2#: . 이 영화들 ...
480,test_481,25,"#Person1#: 자, 우리 계획을 세워야 해. 우선, 너는 어느 침대를 원해?\..."


⚠️ 연속 줄바꿈(\n\n) 포함된 행의 개수
➡️ train_df : 0
➡️ val_df   : 0
➡️ test_df  : 0


In [65]:
#========== dialogue 길이 ==========#
print("📊 dialogue 문자 길이 분포")
print("➡️ train_df")
display(train_df["dialogue_len"].describe())
print("➡️ val_df")
display(val_df["dialogue_len"].describe())
print("➡️ test_df")
display(test_df["dialogue_len"].describe())

#========== dialogue 길이 이상치 확인 ==========#
q1_train = train_df["dialogue_len"].quantile(0.01)          # 하위 1%
q1_val = val_df["dialogue_len"].quantile(0.01)
q1_test = test_df["dialogue_len"].quantile(0.01)
q5_train = train_df["dialogue_len"].quantile(0.05)          # 하위 5%
q5_val = val_df["dialogue_len"].quantile(0.05)
q5_test = test_df["dialogue_len"].quantile(0.05)
q95_train = train_df["dialogue_len"].quantile(0.95)         # 상위 5%
q95_val = val_df["dialogue_len"].quantile(0.95)
q95_test = test_df["dialogue_len"].quantile(0.95)
q99_train = train_df["dialogue_len"].quantile(0.99)         # 상위 1%
q99_val = val_df["dialogue_len"].quantile(0.99)
q99_test = test_df["dialogue_len"].quantile(0.99)

long_dialogues_train = train_df[train_df["dialogue_len"] >= q99_train]
long_dialogues_val = val_df[val_df["dialogue_len"] >= q99_val]
long_dialogues_test = test_df[test_df["dialogue_len"] >= q99_test]
short_dialogues_train = train_df[train_df["dialogue_len"] <= q5_train]
short_dialogues_val = val_df[val_df["dialogue_len"] <= q5_val]
short_dialogues_test = test_df[test_df["dialogue_len"] <= q5_test]

print("\n⚠️ 너무 긴 dialogue 개수")
print("➡️ train_df :", len(long_dialogues_train))
display(long_dialogues_train[["fname", "dialogue", "dialogue_len", "summary"]].head())
print("➡️ val_df   :", len(long_dialogues_val))
display(long_dialogues_val[["fname", "dialogue", "dialogue_len", "summary"]].head())
print("➡️ test_df :", len(long_dialogues_test))
display(long_dialogues_test[["fname", "dialogue", "dialogue_len"]].head())

print("\n⚠️ 너무 짧은 dialogue 개수")
print("➡️ train_df :", len(short_dialogues_train))
display(short_dialogues_train[["fname", "dialogue", "dialogue_len", "summary"]].head())
print("➡️ val_df   :", len(short_dialogues_val))
display(short_dialogues_val[["fname", "dialogue", "dialogue_len", "summary"]].head())
print("➡️ test_df :", len(short_dialogues_test))
display(short_dialogues_test[["fname", "dialogue", "dialogue_len"]].head())

📊 dialogue 문자 길이 분포
➡️ train_df


count    12457.000000
mean       399.410853
std        194.730463
min         83.000000
25%        275.000000
50%        363.000000
75%        492.000000
max       2109.000000
Name: dialogue_len, dtype: float64

➡️ val_df


count     499.000000
mean      393.711423
std       184.239741
min       112.000000
25%       270.000000
50%       360.000000
75%       480.000000
max      1257.000000
Name: dialogue_len, dtype: float64

➡️ test_df


...

,fname,dialogue,dialogue_len,summary
53,train_53,#Person1#: 책 빌려주셔서 감사합니다.\n#Person2#: 괜찮아. 그런데...,146,"#Person1#은 #Person2#에게 책을 빌렸고, 중요한 단어들만 찾아보며 읽..."
56,train_56,#Person1#: 그 영화에서 두 주연 배우의 연기 어떻게 생각해?\n#Perso...,128,"#Person2#는 주인공 연기가 여주인공보다 부족하다고 생각하지만, #Person..."
77,train_77,#Person1#: 기가 밤에 자주 깰 것 같아요.\n#Person2#: 엄마가 이...,104,"#Person2#는 어머니가 이사 오신 후 아기가 밤에 더 잘 자게 되었지만, 본인..."
78,train_78,#Person1#: 뭐 찾고 있어요?\n#Person2#: 19세기 유럽 그림 모음...,137,#Person1#은 #Person2#가 19세기 유럽 그림 모음집을 찾는 것을 도와...
130,train_130,#Person1#: 생각만 해도 신나요! 하와이로 가는 첫 여행이에요. 그런데 거기...,114,"#Person1#은 하와이 여행 중 길을 잃을까 걱정하며, #Person2#은 안내..."


➡️ val_df   : 25


,fname,dialogue,dialogue_len,summary
25,dev_25,"#Person1#: 저기요, 직원분, 걸레 좀 갖다 주실 수 있나요? 와인을 쏟았어...",150,#Person1#이 와인을 쏟아 #Person2#에게 걸레와 종이 냅킨을 요청합니다.
37,dev_37,#Person1#: 뭐 도와드릴까요?\n#Person2#: 여기에 빈 방 있나요?\...,138,"#Person1#은 세 개의 방이 있는 아파트를 추천하지만, #Person2#는 더..."
60,dev_60,#Person1#: 계모가 정말 못됐어. 그 아이가 행복해하는 모습을 보고 참질 못...,150,#Person1#과 #Person2#는 계모가 아이를 괴롭히는 것에 대해 이야기합니다.
66,dev_66,"#Person1#: 저기요, 이 자리 비었나요?\n#Person2#: 비어 있어요....",129,"#Person1#이 자리 공유를 요청하고, #Person2#이 동의합니다."
82,dev_82,#Person1#: 일자리를 찾고 싶어요.\n#Person2#: 여기 여러 파트타임...,133,#Person1#은 컴퓨터 센터의 일자리에 지원하려고 합니다.


➡️ test_df : 26


,fname,dialogue,dialogue_len
19,test_19,#Person1#: 무엇을 도와드릴까요?\n#Person2#: 새 휴대폰을 사고 싶...,127
29,test_29,#Person1#: 혹시 사람들하고 잘 어울리시나요?\n#Person2#: 저는 친...,158
90,test_90,"#Person1#: 엄마, 소풍에 가져갈 과일 있어요? 오렌지랑 바나나가 몇 개 있...",156
123,test_123,#Person1#: 저희랑 어떻게 연락하면 될까요?\n#Person2#: 전화나 이...,137
138,test_138,"#Person1#: 자기야, 잠깐 나 좀 도와줄 수 있어?\n#Person2#: 물...",146


In [66]:
#========== 이상치를 가지는 행의 dialogue 직접 살펴보기 ==========#

print("🔎 이상치를 가지는 행의 dialogue 직접 살펴보기")

k=210
print("✅ dialogue")
print(train_df.loc[k]["dialogue"])
print("\n✅ summary")
print(train_df.loc[k]["summary"])

# 진짜 한두 문장만 있는 대화인지
# summary는 꽤 긴데 dialogue가 너무 짧아서 조건-정답 미스매치처럼 보이는지
# 완전 이상한 샘플(오타 / 반쯤 잘린 데이터)이 있는지
# 이거 확인하려면 따로 .csv로 저장해야하려나?

🔎 이상치를 가지는 행의 dialogue 직접 살펴보기
✅ dialogue
#Person1#: 앨버트랑 저는 집에 필요한 물건들이 많아요. 그런데 어디에서 사야 가장 좋은지 잘 모르겠네요. 조언 좀 해줄 수 있을까요?
#Person2#: 물론이에요, Carolyn. 도와드릴 수 있어서 기뻐요. 지금 어떤 물건들이 필요한가요?
#Person1#: 가구는 대부분 있는데, 주방 용품이랑 욕실 소품, 청소랑 세탁 관련 소품들이 필요해요. 앨버트는 작은 사무실도 꾸미고 싶어해요.
#Person2#: 앨버트라면 사무용품 가게에 가는 게 좋겠네요. Office Depot라는 매장이 있어요. 필요한 건 다 있을 거예요. 가격도 괜찮고요. 위치도 편리하고요.
#Person1#: 적어도 될까요?
#Person2#: 물론이죠. Office, 그리고 D - E - P - O - T 이렇게 써요. Yellow Pages에 나와 있을 거예요.
#Person1#: 고마워요.
#Person2#: 주방 용품은 높은 품질을 원하세요?
#Person1#: 그게 무슨 뜻이죠?
#Person2#: 최상의 품질을 원하시는지, 아니면 좋은 가격을 원하시는지요?
#Person1#: 마도 좋은 가격이요. 저희는 미국에 1년 정도밖에 안 있을 거거든요.
#Person2#: 그러면 K-Mart 아니면 Wal-Mart에 가보세요.
#Person1#: 그 가게들은 뭐죠?
#Person2#: 주 큰 할인 백화점이에요. 그래서 가격이 정말 좋아요. 주방이나 욕실에 필요한 모든 걸 찾을 수 있을 거예요.
#Person1#: 심지어 은식기도요?
#Person2#: 모든 게. 가장 비싼 브랜드는 없지만, 품질은 대체로 괜찮아요. 중요한 건, 가격이 좋고 아주 편리하다는 거죠.
#Person1#: Price Club이란 곳에 대해서 친구가 얘기하던데요. 거기 대해 아세요?
#Person2#: 거기서 쇼핑해본 적은 없지만, Price Club은 회원제 매장으로 알고 있어요.
#Person1#: 그게 무슨 뜻이죠?
#Pers

In [67]:
# encoder_max_length와 비교
encoder_max_len = loaded_config["tokenizer"]["encoder_max_len"]
print(f"📌 encoder_max_length 설정값 : {encoder_max_len}")

ratio_over_train = (train_df["dialogue_len_token"] > encoder_max_len).mean()
ratio_over_val = (val_df["dialogue_len_token"] > encoder_max_len).mean()
ratio_over_test = (test_df["dialogue_len_token"] > encoder_max_len).mean()

print("\n⚠️ encoder_max_length를 넘는 dialogue 비율")
print("    - train_df :", f"{ratio_over_train*100:.2f}%")
print("    - val_df   :", f"{ratio_over_val*100:.2f}%")
print("    - test_df  :", f"{ratio_over_test*100:.2f}%")

# 토큰 길이 분포
print("\n✅ train_df")
print("📊 dialogue 토큰 길이 분포")
display(train_df["dialogue_len_token"].describe())

print("🔎 토큰 길이 (상위 5개)")
display(train_df.sort_values("dialogue_len_token", ascending=False).head()[["fname", "dialogue_len_token", "dialogue"]])
print("🔎 토큰 길이 (하위 5개)")
display(train_df.sort_values("dialogue_len_token", ascending=False).tail()[["fname", "dialogue_len_token", "dialogue"]])


print("\n✅ val_df")
print("📊 dialogue 토큰 길이 분포")
display(val_df["dialogue_len_token"].describe())

print("🔎 토큰 길이 (상위 5개)")
display(val_df.sort_values("dialogue_len_token", ascending=False).head()[["fname", "dialogue_len_token", "dialogue"]])
print("🔎 토큰 길이 (하위 5개)")
display(val_df.sort_values("dialogue_len_token", ascending=False).tail()[["fname", "dialogue_len_token", "dialogue"]])


print("\n✅ test_df")
print("📊 dialogue 토큰 길이 분포")
display(test_df["dialogue_len_token"].describe())

print("🔎 토큰 길이 (상위 5개)")
display(test_df.sort_values("dialogue_len_token", ascending=False).head()[["fname", "dialogue_len_token", "dialogue"]])
print("🔎 토큰 길이 (하위 5개)")
display(test_df.sort_values("dialogue_len_token", ascending=False).tail()[["fname", "dialogue_len_token", "dialogue"]])

📌 encoder_max_length 설정값 : 512

⚠️ encoder_max_length를 넘는 dialogue 비율
    - train_df : 0.00%
    - val_df   : 0.00%
    - test_df  : 0.00%

✅ train_df
📊 dialogue 토큰 길이 분포


count    12457.000000
mean        83.186401
std         43.133518
min         17.000000
25%         55.000000
50%         74.000000
75%        104.000000
max        496.000000
Name: dialogue_len_token, dtype: float64

🔎 토큰 길이 (상위 5개)


,fname,dialogue_len_token,dialogue
7368,train_7368,496,"#Person1#: 안녕, 어떻게 지내?\n#Person2#: 뭐 그냥 그럭저럭....."
6178,train_6178,457,"#Person1#: 그럼 시작해볼까요? 먼저 당신에 대해 알려주시고, 배경에 대해 ..."
2906,train_2906,456,"#Person1#: 안녕하세요, 저희 새 프로그램 Exotic Animal King..."
11708,train_11711,436,#Person1#: 그럼 일단 본인과 본인의 배경에 대해 말씀해 주시겠어요?\n#P...
4724,train_4724,427,#Person1#: 안녕하세요. 어떻게 도와드릴까요?\n#Person2#: 중간 크...


🔎 토큰 길이 (하위 5개)


...

count    499.000000
mean      86.609218
std       46.849087
min       22.000000
25%       53.000000
50%       77.000000
75%      109.500000
max      455.000000
Name: dialogue_len_token, dtype: float64

🔎 토큰 길이 (상위 5개)


,fname,dialogue_len_token,dialogue
434,test_434,455,"#Person1#: 안녕, Andrew! 안녕 ...? Andrew.\n#Perso..."
87,test_87,342,"#Person1#: 좋은 아침이에요, Ms. Ross. 들어오세요.\n#Person..."
483,test_484,253,#Person1#: 저기요.\n#Person2#: ?\n#Person1#: 여기 처...
118,test_118,252,#Person1#: 저기요. 이 기계들 어떻게 사용해야 하는지 좀 알려줄 수 있나요...
480,test_481,236,"#Person1#: 자, 우리 계획을 세워야 해. 우선, 너는 어느 침대를 원해?\..."


🔎 토큰 길이 (하위 5개)


,fname,dialogue_len_token,dialogue
123,test_123,27,#Person1#: 저희랑 어떻게 연락하면 될까요?\n#Person2#: 전화나 이...
333,test_333,26,"#Person1#: 나 진짜 짜증나. 과제를 컴퓨터로 해야 하는데, 도서관에서 컴퓨..."
185,test_185,26,#Person1#: 저기요. Mr. Lee 찾고 있는데요. 이거 그분 소포예요.\n...
357,test_357,24,"#Person1#: 안녕하세요, 도와드릴까요?\n#Person2#: 예약했어요.\n..."
19,test_19,22,#Person1#: 무엇을 도와드릴까요?\n#Person2#: 새 휴대폰을 사고 싶...


In [68]:
#========== 각 줄의 좌우 공백 존재 여부 ==========#
print("🔎 줄 앞/뒤 공백이 있는 dialogue 개수")
print("➡️ train_df :", train_df["d_has_space_edges"].sum())
print(train_df["d_has_space_edges"].value_counts())
print("➡️ val_df   :", val_df["d_has_space_edges"].sum())
print(val_df["d_has_space_edges"].value_counts())
print("➡️ test_df  :", test_df["d_has_space_edges"].sum())
print(test_df["d_has_space_edges"].value_counts())

#========== 연속 공백(2칸 이상) 여부 ==========#
print("\n🔎 연속 공백(2칸 이상)이 존재하는 dialogue 수")
print("➡️ train_df :", train_df["d_has_multi_spaces"].sum())
print(train_df["d_has_multi_spaces"].value_counts())
print("➡️ val_df :", val_df["d_has_multi_spaces"].sum())
print(val_df["d_has_multi_spaces"].value_counts())
print("➡️ test_df :", test_df["d_has_multi_spaces"].sum())
print(test_df["d_has_multi_spaces"].value_counts())

# #========== 실제 컬럼 눈으로 보기 ==========#
# print(train_df[train_df["has_space_edges"]]["dialogue"])
# train_df[train_df["has_space_edges"]].loc[11]["dialogue"]
# print(train_df[train_df["has_multi_spaces"]]["dialogue"])
# train_df[train_df["has_multi_spaces"]].loc[177]["dialogue"]

🔎 줄 앞/뒤 공백이 있는 dialogue 개수
➡️ train_df : 6
d_has_space_edges
False    12451
True         6
Name: count, dtype: int64
➡️ val_df   : 2
d_has_space_edges
False    497
True       2
Name: count, dtype: int64
➡️ test_df  : 0
d_has_space_edges
False    499
Name: count, dtype: int64

🔎 연속 공백(2칸 이상)이 존재하는 dialogue 수
➡️ train_df : 0
d_has_multi_spaces
False    12457
Name: count, dtype: int64
➡️ val_df : 0
d_has_multi_spaces
False    499
Name: count, dtype: int64
➡️ test_df : 0
d_has_multi_spaces
False    499
Name: count, dtype: int64


In [69]:
# def normalize_spaces(dialogue: str) -> str:
#     # 줄마다 앞뒤 공백 제거
#     lines = [ln.strip() for ln in dialogue.split("\n")]
#     text = "\n".join(lines)
#     # 내부에서 2칸 이상 공백 → 1칸
#     text = re.sub(r"[ \t]{2,}", " ", text)
#     return text

# train_df_preview = train_df.copy()
# train_df_preview["dialogue_clean"] = train_df_preview["dialogue"].apply(
#     lambda x: normalize_spaces(x, drop_smalltalk=False)  # 일단 제거는 False로
# )

##### Dialogue Noise 분석

In [70]:
# # dialogue noise 시각적 EDA | noise : dialogue에 있지만 summary에는 사용되지 않은 것
#     # 정보 요약 중심 스타일
#     # 상황, 주요 행동/사실, 결과/다음 조치 > 중심 요약 대상
#     # 감정·감탄·상대적 표현·반응성 문장 > 공격적 제거 대상

# print("✅ train_df.shape: ", train_df.shape)

# n=150
# print("\n🔎 dialogue")
# print(train_df.iloc[n]["dialogue"])

# print("\n🔎 summary")
# print(train_df.iloc[n]["summary"])

In [71]:
#========== Noise 패턴 정의 ==========#

SYMBOL_PATTERNS = {
    "dots": r"\.{2,}",              # .., ... 등
    "tilde": r"~+",
    "exclam": r"!{2,}",
    "question": r"\?{2,}",          # 같은 기호 2번 반복
    "repeat_kor": r"[ㄱ-ㅎ]{2,}",   # ㅋㅋ, ㅎㅎ, ㅠㅠ 등
    "stars": r"\*+",
}

NOISE_PATTERNS_TIER1 = {
    # 1) 감탄사 + 특수기호 (아..., 어?!, 음~, 으… 등)
    "filler_with_punct": r"(아+|어+|음+|으+)[\.…~!\?]+",

    # 2) 연속 감탄사 (아아, 어어, 음음, 으으)
    "repeat_filler": r"(아|어|음|으){2,}",

    # 3) 울음 (ㅠㅠ, ㅜㅜ, ㅠㅜㅜ 이런 것들)
    "cry": r"[ㅠㅜ]{2,}",

    # 4) 웃음 (ㅋㅋㅋ, ㅎㅎㅎ)
    "laugh_kek": r"ㅋ{2,}",
    "laugh_heh": r"ㅎ{2,}",
    
    # 5) 자음 반복 (ㄱㄱ, ㅎㅎ, ㅋㅋ 등 — 이미 위에 걸리는 애도 있지만 여유 버퍼)
    "repeat_jamo": r"[ㄱ-ㅎ]{2,}",
}

NOISE_PATTERNS_TIER2 = {
    # 1) 화자 태그 뒤에 바로 나오는 단발 filler (아, 어, 음, 으, 응, 네)
    # 예: "#Person2#: 아, 그냥 피곤해." → "그냥 피곤해."
    "leading_filler_after_tag": r"(?m)^#Person\d+#:\s(?:아|어|음|으|응|네)[, ]+",

    # 2) 화자 태그 뒤에서 단독 감탄사만 말하는 경우
    # 예: "#Person1#: 와!" / "#Person1#: 어머!" → 통째로 제거
    "solo_exclaim_after_tag": r"(?m)^#Person\d+#:\s*(와|오|어머|헐)[! ]*$",

    # 3) 인사 / 스몰토크 위주 발화
    # 예: "안녕, 메리. 잘 지내고 있어?", "안녕하세요." 등
    "greetings_after_tag": (
        r"(?m)^#Person\d+#:\s*"
        r"(안녕|안녕하세요|여보세요|좋은 아침이야|잘 지내[요]?)"
        r"[^\.!?]*[\.!?]?$"
    ),

    # 4) 반응형 짧은 의문 (정말?, 진짜?)
    # 예: "#Person1#: 정말?", "#Person2#: 진짜?"
    "reaction_questions_after_tag": (
        r"(?m)^#Person\d+#:\s*(정말|정말로|진짜)\?[ \t]*$"
    ),

    # 5) 문장 전체가 "긍정/동의/감탄" 반응인 경우
    # 예: "그럼요.", "좋죠.", "좋아요.", "좋습니다.", "물론이지.", "글쎄요."
    "sentence_reactions": (
        r"(?m)^#Person\d+#:\s*"
        r"(그럼요|좋죠|좋아요|좋습니다|물론이지|글쎄요|그렇구나|그런 것 같아|"
        r"그럴 수도 있어|그럴 수도 있을 것 같아|필요 없을 것 같아|좋지)"
        r"[\.!? ]*$"
    ),

    # 6) 가벼운 반응/감정 의견 (정보 없이 감정/취향만)
    # 예: "신기하네요.", "좋네요.", "정말 아름다운 집이야.", "가끔 일찍 집에 가는 것도 좋지."
    "emotive_reactions": (
        r"(?m)^#Person\d+#:\s*"
        r"(신기하네요|좋네요|정말 아름다운.*|좋은 느낌.*|이해하기 어려워|"
        r"취향이 아니니까|가끔 .*좋지)[\.!? ]*$"
    ),

    # 7) 짧은 in-line filler (문장 안/혼자 등장하는 경우)
    # 예: "잠시만요.", "어쨌든,", "그럴 수도 있고,", "제가 의사는 아니지만,"
    "inline_fillers": r"(잠시만요|어쨌든|그럴 수도 있고|제가 의사는 아니지만)",
}

NOISE_PATTERNS_TIER3 = {
    # 1) 짧은 반응성 발화 — 그래/맞아/응/알겠어 (내용 기여 거의 없음)
    # 예: "#Person2#: 그래.", "#Person2#: 응, 맞아."
    "short_reactions": (
        r"(?m)^#Person\d+#:\s*"
        r"(그래[요]?|맞아|응|알겠어|알겠습니다)[\.!?]?$"
    ),

    # 2) 감사/사과/예의 표현 전반
    # 예: "고마워요.", "정말 감사합니다.", "죄송합니다.", "안녕히 계세요.", "행운을 빌어요."
    "polite_phrases": (
        r"(고마워(?:요)?|정말 고마워요|감사합니다!?|정말 감사합니다|감사해요|"
        r"미안해(?:요)?|죄송합니다|죄송하지만|안녕히 계세요|행운을 빌(?:어|어요)?|"
        r"천만에요)"
        r"안녕|안녕하세요|어서 오세요|반가워|좋은 아침|굿모닝"
        r"고마워|감사|수고했|잘 지내|날씨|오늘 어때"
    ),

    # 3) small-talk / 근황 물어보기류 — 요약에서 거의 안 쓰임
    # 예: "부모님은 잘 지내셔?", "어디 가는 길이야?", "대화가 즐거웠습니다."
    "small_talk": (
        r"(부모님은 잘 지내셔|어디 가는 길|근황[은] 어때|대화가 즐거웠습니다|얘기해서 좋았어)"
    ),

    # 4) 반복적 확인 질문 (내용 없이 절차만 묻는 것)
    # 예: "확실하세요?", "무게 초과인가요?", "~할 수 있을까요?"
    "procedural_questions": (
        r"(확실하세요\?|무게 초과인가요\?|[가-힣\s]*가능할까요\?)"
    ),

    # 5) 서비스/접대/절차 안내 문장
    # 예: "도와드릴까요?", "이쪽으로 오세요.", "앉으세요.", "짐 좀 옮길게요.", "무료 주스 가져다 드릴게요."
    "service_phrases": (
        r"(도와드릴까요|이쪽으로 오세요|앉으세요|앉아도 돼|짐 좀 옮길게요|"
        r"무료 주스|음료 카트|카트 하나 받아요|저쪽에 있어요|왼쪽으로 가면 됩니다|"
        r"직진하면.*길이에요|환영합니다|계속 열심히 하세요)"
    ),

    # 6) 방송·아나운서·메타 표현
    # 예: "소비자 보고서를 마무리하겠습니다.", "스포츠 팬 여러분", "넘기겠습니다."
    "meta_broadcast": (
        r"(소비자 보고서를.*마무리|넘기겠습니다|스포츠 팬 여러분)"
    ),

    # 화자태그 없는 10자 이내 한 줄
    "short_excl_pat" : (
        r"^[^#]{0,10}$"
    ),
}

In [72]:
# 특수문자/기호 패턴 찾는 함수
def count_pattern_series(series, pattern):
    return series.str.count(pattern)

# 실행
print("📊 특수문자/기호 패턴별 비율")
for name, p in SYMBOL_PATTERNS.items():
    cnt = count_pattern_series(train_df["dialogue"], p).sum()
    print(f"- {name:10s} : {cnt} 회 | {p}")

# # 패턴별 샘플 확인 함수
# def show_samples_for_pattern(df, pattern, n=5, col="dialogue"):
#     mask = df[col].str.contains(pattern, regex=True)
#     print(f"\n✅ 패턴 [{pattern}] 이 포함된 샘플 {n}개")
#     display(df.loc[mask, ["fname", col]].head(n))

# for name, p in SYMBOL_PATTERNS.items():
#     show_samples_for_pattern(train_df, p, n=3)

📊 특수문자/기호 패턴별 비율
- dots       : 1690 회 | \.{2,}
- tilde      : 45 회 | ~+


- exclam     : 0 회 | !{2,}
- question   : 0 회 | \?{2,}
- repeat_kor : 1 회 | [ㄱ-ㅎ]{2,}
- stars      : 1 회 | \*+


In [73]:
# 감탄사·불필요한 표현 패턴 분석
def collect_noise_phrases(text: str, patterns) -> list[tuple[str, str]]:
    found = []

    # patterns 타입에 따라 반복 대상 정하기
    if isinstance(patterns, dict):
        iterable = patterns.items()      # ("이름", "패턴")
    else:
        iterable = patterns              # [("이름", "패턴"), ...]

    for name, pattern in iterable:
        matches = re.findall(pattern, text)
        if not matches:
            continue

        for m in matches:
            # 정규식에 그룹이 있으면 m이 tuple일 수 있어서 flatten
            if isinstance(m, tuple):
                m = "".join(m)
            found.append((name, m))

    return found

In [74]:
#========== NOISE_PATTERNS_TIER1 ==========#

all_noise_01_train = []
all_noise_01_val = []
all_noise_01_test = []

for s in train_df["dialogue"]:
    all_noise_01_train.extend(collect_noise_phrases(s, NOISE_PATTERNS_TIER1))
for t in val_df["dialogue"]:
    all_noise_01_val.extend(collect_noise_phrases(t, NOISE_PATTERNS_TIER1))
for u in test_df["dialogue"]:
    all_noise_01_test.extend(collect_noise_phrases(u, NOISE_PATTERNS_TIER1))

noise_01_df_train = pd.DataFrame(all_noise_01_train, columns=["type", "token"])
noise_01_df_val = pd.DataFrame(all_noise_01_val, columns=["type", "token"])
noise_01_df_test = pd.DataFrame(all_noise_01_test, columns=["type", "token"])

print("➡️ train_df")
print("📊 NOISE_PATTERNS_TIER1 | noise type 분포")
display(noise_01_df_train["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER1 | 각 noise type별 토큰 예시")
display(noise_01_df_train.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ val_df")
print("📊 NOISE_PATTERNS_TIER1 | noise type 분포")
display(noise_01_df_val["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER1 | 각 noise type별 토큰 예시")
display(noise_01_df_val.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ test_df")
print("📊 NOISE_PATTERNS_TIER1 | noise type 분포")
display(noise_01_df_test["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER1 | 각 noise type별 토큰 예시")
display(noise_01_df_test.groupby("type")["token"].value_counts().groupby(level=0).head())

➡️ train_df
📊 NOISE_PATTERNS_TIER1 | noise type 분포


type
filler_with_punct    38293
repeat_filler          112
laugh_heh                1
repeat_jamo              1
cry                      1
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER1 | 각 noise type별 토큰 예시


type               token
cry                ㅠㅠ           1
filler_with_punct  어        25532
                   아        12741
                   음           14
                   아아           3
                   으            2
laugh_heh          ㅎㅎ           1
repeat_filler      으           76
                   어           15
                   음           12
                   아            9
repeat_jamo        ㅎㅎ           1
Name: count, dtype: int64

➡️ val_df
📊 NOISE_PATTERNS_TIER1 | noise type 분포


...

type               token
filler_with_punct  어        1071
                   아         483
                   아아          1
                   음           1
repeat_filler      으           4
                   어           2
                   아           1
Name: count, dtype: int64

➡️ test_df
📊 NOISE_PATTERNS_TIER1 | noise type 분포


type
filler_with_punct    1574
repeat_filler           4
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER1 | 각 noise type별 토큰 예시


type               token
filler_with_punct  어        1059
                   아         509
                   음           6
repeat_filler      으           3
                   어           1
Name: count, dtype: int64

In [75]:
#========== NOISE_PATTERNS_TIER2 ==========#

all_noise_02_train = []
all_noise_02_val = []
all_noise_02_test = []

for s in train_df["dialogue"]:
    all_noise_02_train.extend(collect_noise_phrases(s, NOISE_PATTERNS_TIER2))
for t in val_df["dialogue"]:
    all_noise_02_val.extend(collect_noise_phrases(t, NOISE_PATTERNS_TIER2))
for u in test_df["dialogue"]:
    all_noise_02_test.extend(collect_noise_phrases(u, NOISE_PATTERNS_TIER2))

noise_02_df_train = pd.DataFrame(all_noise_02_train, columns=["type", "token"])
noise_02_df_val = pd.DataFrame(all_noise_02_val, columns=["type", "token"])
noise_02_df_test = pd.DataFrame(all_noise_02_test, columns=["type", "token"])

print("➡️ train_df")
print("📊 NOISE_PATTERNS_TIER2 | noise type 분포")
display(noise_02_df_train["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER2 | 각 noise type별 토큰 예시")
display(noise_02_df_train.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ val_df")
print("📊 NOISE_PATTERNS_TIER2 | noise type 분포")
display(noise_02_df_val["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER2 | 각 noise type별 토큰 예시")
display(noise_02_df_val.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ test_df")
print("📊 NOISE_PATTERNS_TIER2 | noise type 분포")
display(noise_02_df_test["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER2 | 각 noise type별 토큰 예시")
display(noise_02_df_test.groupby("type")["token"].value_counts().groupby(level=0).head())

➡️ train_df
📊 NOISE_PATTERNS_TIER2 | noise type 분포


type
greetings_after_tag             1047
inline_fillers                   388
sentence_reactions               194
leading_filler_after_tag         157
reaction_questions_after_tag      92
emotive_reactions                 42
solo_exclaim_after_tag             7
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER2 | 각 noise type별 토큰 예시


type                          token                      
emotive_reactions             좋네요                             31
                              정말 아름다운 노을이야!                    1
                              정말 아름다운 도시예요. 분명히 좋아하실 거예요.      1
                              정말 아름다운 드라이브예요.                  1
                              정말 아름다운 배네! 이건 누구야?              1
greetings_after_tag           안녕                             887
                              여보세요                           121
                              잘 지내                            30
                              좋은 아침이야                          6
                              잘 지내요                            3
inline_fillers                잠시만요                           233
                              어쨌든                            154
                              그럴 수도 있고                         1
leading_filler_after_tag      #Person2#: 네,                   57
                              #P

➡️ val_df
📊 NOISE_PATTERNS_TIER2 | noise type 분포


...

type                          token         
emotive_reactions             정말 아름다운 집이야!       1
                              좋네요                1
greetings_after_tag           안녕                27
                              여보세요               6
                              잘 지내               1
inline_fillers                어쨌든               10
                              잠시만요               6
                              그럴 수도 있고           1
leading_filler_after_tag      #Person2#: 음       1
                              #Person2#: 으,      1
                              #Person1#: 네,      1
                              #Person1#: 네       1
                              #Person2#: 네,      1
reaction_questions_after_tag  정말                 1
                              정말로                1
sentence_reactions            좋아요                2
                              물론이지               2
                              그렇구나               1
                              그럴 수도 있

➡️ test_df
📊 NOISE_PATTERNS_TIER2 | noise type 분포


type
greetings_after_tag             43
sentence_reactions              14
inline_fillers                  14
leading_filler_after_tag        10
emotive_reactions                4
reaction_questions_after_tag     3
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER2 | 각 noise type별 토큰 예시


type                          token                                                                                    
emotive_reactions             좋네요                                                                                           3
                              정말 아름다운 도시야, 아주 오래된 도시지. 많은 오래된 건물들이 잘 복원되어 있어. 17세기와 18세기에 지어진 건물들도 있어. 거기 가면 정말 좋아할 거야.     1
greetings_after_tag           안녕                                                                                           38
                              여보세요                                                                                          4
                              잘 지내                                                                                          1
inline_fillers                잠시만요                                                                                          9
                              어쨌든                                                                                           

In [76]:
#========== NOISE_PATTERNS_TIER3 ==========#

all_noise_03_train = []
all_noise_03_val = []
all_noise_03_test = []

for s in train_df["dialogue"]:
    all_noise_03_train.extend(collect_noise_phrases(s, NOISE_PATTERNS_TIER3))
for t in val_df["dialogue"]:
    all_noise_03_val.extend(collect_noise_phrases(t, NOISE_PATTERNS_TIER3))
for u in test_df["dialogue"]:
    all_noise_03_test.extend(collect_noise_phrases(u, NOISE_PATTERNS_TIER3))

noise_03_df_train = pd.DataFrame(all_noise_03_train, columns=["type", "token"])
noise_03_df_val = pd.DataFrame(all_noise_03_val, columns=["type", "token"])
noise_03_df_test = pd.DataFrame(all_noise_03_test, columns=["type", "token"])

print("➡️ train_df")
print("📊 NOISE_PATTERNS_TIER3 | noise type 분포")
display(noise_03_df_train["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER3 | 각 noise type별 토큰 예시")
display(noise_03_df_train.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ val_df")
print("📊 NOISE_PATTERNS_TIER3 | noise type 분포")
display(noise_03_df_val["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER3 | 각 noise type별 토큰 예시")
display(noise_03_df_val.groupby("type")["token"].value_counts().groupby(level=0).head())

print("➡️ test_df")
print("📊 NOISE_PATTERNS_TIER3 | noise type 분포")
display(noise_03_df_test["type"].value_counts())
print("\n🔎 NOISE_PATTERNS_TIER3 | 각 noise type별 토큰 예시")
display(noise_03_df_test.groupby("type")["token"].value_counts().groupby(level=0).head())

➡️ train_df
📊 NOISE_PATTERNS_TIER3 | noise type 분포


type
polite_phrases          6095
service_phrases         1606
short_reactions          426
procedural_questions      55
small_talk                 7
meta_broadcast             2
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER3 | 각 noise type별 토큰 예시


type                  token         
meta_broadcast        넘기겠습니다               2
polite_phrases                          6095
procedural_questions   가능할까요?             10
                       혹시 할인 가능할까요?        2
                       신분증 확인 가능할까요?       1
                       화요일은 가능할까요?         1
                      분에 가능할까요?            1
service_phrases       도와드릴까요            1180
                      환영합니다              206
                      앉으세요               138
                      이쪽으로 오세요            67
                      저쪽에 있어요             11
short_reactions       알겠습니다              213
                      맞아                 127
                      알겠어                 63
                      그래                  16
                      그래요                  7
small_talk            어디 가는 길              4
                      얘기해서 좋았어             2
                      대화가 즐거웠습니다           1
Name: count, dtype: int64

➡️ val_df
📊 NOISE_PATTERNS_TIER3 | noise type 분포


...

type                  token                  
meta_broadcast        넘기겠습니다                       2
                      스포츠 팬 여러분                    1
                      소비자 보고서를 여기서 마무리             1
polite_phrases                                   241
procedural_questions   그럼 가장 빨리 언제 선적이 가능할까요?      1
                      무게 초과인가요?                    1
                      월 중에 선적이 가능할까요?              1
                      월에 선적이 가능할까요?                1
                      확실하세요?                       1
service_phrases       도와드릴까요                      42
                      앉으세요                         7
                      환영합니다                        4
                      저쪽에 있어요                      2
                      이쪽으로 오세요                     2
short_reactions       맞아                           7
                      알겠습니다                        6
                      알겠어                          4
                      그래                           1


➡️ test_df
📊 NOISE_PATTERNS_TIER3 | noise type 분포


type
polite_phrases     233
service_phrases     59
short_reactions     16
Name: count, dtype: int64


🔎 NOISE_PATTERNS_TIER3 | 각 noise type별 토큰 예시


type             token   
polite_phrases               233
service_phrases  도와드릴까요       45
                 환영합니다         8
                 이쪽으로 오세요      4
                 앉으세요          2
short_reactions  알겠습니다         6
                 맞아            5
                 알겠어           3
                 그래            2
Name: count, dtype: int64

In [77]:
# 여기서 noise 중에 빈도가 높은 것,
# 그리고 그게 summary에 나타나는 빈도를 확인한 후에 처리할 수 있을 듯

### Summary

In [78]:
# 화자 역할(#Person1#, #Person2#) 등장 규칙
print("✅ train_df")
print("📊 summary 내 화자 태그 존재 여부")
print(pd.crosstab(train_df["s_has_p1"], train_df["s_has_p2"], rownames=["#Person1#"], colnames=["#Person2#"]))

print("\n📊 각 패턴 비율") # 두 사람의 대화가 대부분인 듯
ct = pd.crosstab(train_df["s_has_p1"], train_df["s_has_p2"], normalize=True)
print((ct * 100).round(2))

print("\n✅ val_df")
print("📊 summary 내 화자 태그 존재 여부")
print(pd.crosstab(val_df["s_has_p1"], val_df["s_has_p2"], rownames=["#Person1#"], colnames=["#Person2#"]))

print("\n📊 각 패턴 비율") # 두 사람의 대화가 대부분인 듯
ct = pd.crosstab(val_df["s_has_p1"], val_df["s_has_p1"], normalize=True)
print((ct * 100).round(2))

print("\n🔎 #Person1#, #Person2# 둘 다 등장하는 예시")
print("✅ train_df")
display(train_df[(train_df["s_has_p1"]) & (train_df["s_has_p2"])].head()[["fname", "summary"]])
print("\n✅ val_df")
display(val_df[(val_df["s_has_p1"]) & (val_df["s_has_p2"])].head()[["fname", "summary"]])

print("\n🔎 #Person1#만 등장하는 예시")
print("✅ train_df")
display(train_df[(train_df["s_has_p1"]) & (~train_df["s_has_p2"])].head()[["fname", "summary"]])
print("\n✅ val_df")
display(val_df[(val_df["s_has_p1"]) & (~val_df["s_has_p2"])].head()[["fname", "summary"]])

print("\n🔎 #Person2#만 등장하는 예시")
print("✅ train_df")
display(train_df[(~train_df["s_has_p1"]) & (train_df["s_has_p2"])].head()[["fname", "summary"]])
print("\n✅ val_df")
display(val_df[(~val_df["s_has_p1"]) & (val_df["s_has_p2"])].head()[["fname", "summary"]])

✅ train_df
📊 summary 내 화자 태그 존재 여부
#Person2#  False  True 
#Person1#              
False       1760    602
True        1917   8178

📊 각 패턴 비율
s_has_p2  False  True 
s_has_p1              
False     14.13   4.83
True      15.39  65.65

✅ val_df
📊 summary 내 화자 태그 존재 여부
#Person2#  False  True 
#Person1#              
False         59     25
True          69    346

📊 각 패턴 비율
s_has_p1  False  True 
s_has_p1              
False     16.83   0.00
True       0.00  83.17

🔎 #Person1#, #Person2# 둘 다 등장하는 예시
✅ train_df


,fname,summary
2,train_2,#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라...
3,train_3,#Person1#은 #Person2#가 여자친구가 있고 결혼할 예정이라는 사실을 말...
6,train_6,#Person1#이 #Person2#에게 택시 요금을 설명합니다.
8,train_8,"#Person1#은 #Person2#에게 기본 컴퓨터 패키지를 설명하며, #Pers..."
9,train_9,"#Person1#은 비자 담당자와의 인터뷰를 앞두고 있으며, #Person2#에게 ..."



✅ val_df


,fname,summary
2,dev_2,"#Person1#은 건강에 안 좋은 음식을 그만 먹기로 결심하고, #Person2#..."
3,dev_3,#Person2#는 UFO를 믿고 꿈에서 볼 수 있다고 한다. #Person1#은 ...
4,dev_4,"#Person1#은 오늘 학교에 가지 않았고, #Person2#는 내일 학교 대신 ..."
5,dev_5,"#Person1#은 #Person2#에게 건강을 위해 담배를 끊을 것을 권유하고, ..."
8,dev_8,"#Person1#은 #Person2#에게 취미를 묻고, #Person2#는 사진 찍..."



🔎 #Person1#만 등장하는 예시
✅ train_df


...

,fname,summary
1,dev_1,#Person1#는 Jimmy를 운동하러 초대하고 팔과 복근 운동을 하도록 설득합니다.
7,dev_7,"#Person1#은 Karen에게 해변에서 주말을 어떻게 보냈는지, 어디에서 머물렀..."
10,dev_10,"#Person1#은 어머니의 과보호로 인해 지쳤고, 독립하여 자신의 꿈을 이루고 싶..."
12,dev_12,Mr. Zhang은 또다시 거절당한 후 #Person1#에게 조언을 구합니다. #P...
17,dev_17,#Person1#은 Sara에게 여행 준비물 몇 가지를 챙기라고 상기시킨다. Sar...



🔎 #Person2#만 등장하는 예시
✅ train_df


,fname,summary
30,train_30,"#Person2#는 맥주 두 병, 오늘의 스페셜인 스테이크와 곁들일 완두콩과 당근,..."
39,train_39,#Person2#와 #Person2#가 아르망의 집에 방문하여 함께 저녁 식사를 합...
44,train_44,"#Person2#는 멋진 생일 선물을 원하지만, 엄마는 여자아이에게 어울릴 만한 선..."
47,train_47,#Person2#는 Carla에게 아파트는 요리와 청소에 많은 시간을 소비하게 되므...
49,train_49,#Person2#는 대학교에서 선전부 장관을 맡았고 농구 활동에 참여했다고 말합니다.



✅ val_df


,fname,summary
0,dev_0,#Person2#는 숨쉬기 어려워합니다. 의사는 #Person2#에게 증상을 확인하...
44,dev_44,#Person2#와 Gary는 오해로 인해 영화 시작 부분을 놓쳤습니다. Gary가...
45,dev_45,"Daniel은 매니저 직책에 지원하며, #Person2#와의 인터뷰에서 회사에 대해..."
57,dev_57,"#Person2#는 저녁 식사를 즐겼으며, 디저트로 아이스크림을 요청했습니다."
78,dev_78,"#Person2#는 몸이 아파 시험을 보지 못했지만, 보충 시험을 볼 수 있게 되었..."


In [79]:
#========== 화자 태그 사용되는 예시 눈으로 보기 ==========#
print("🔎 화자 태그 사용 예시 보기")

m=30
print('✅ dialogue')
print(train_df.loc[m]["dialogue"])
print('\n✅ summary')
print(train_df.loc[m]["summary"])
# 근데 어디에서는 #Person1#, 어디에서는 Judy Liao 이런 식으로 이름이 매칭되는데 이런 건 어떻게 해야 하나?

🔎 화자 태그 사용 예시 보기
✅ dialogue
#Person1#: 주문하시겠어요?
#Person2#: 직 결정을 못 했어요. 조금만 더 시간을 주시겠어요?
#Person1#: 천천히 결정하세요.
#Person2#: 료 좀 시킬 수 있을까요? 맥주 두 병 주세요.
#Person1#: 알겠습니다.
#Person2#: 오늘의 스페셜 메뉴가 뭔지 알려 주세요.
#Person1#: 오늘의 스페셜은 스테이크입니다.
#Person2#: 그럼 스테이크로 부탁할게요.
#Person1#: 스테이크에는 어떤 것을 곁들이시겠어요?
#Person2#: 완두콩과 당근으로 주세요.
#Person1#: 알겠습니다. 디저트는 무엇으로 하시겠어요?
#Person2#: 이스크림 주세요.

✅ summary
#Person2#는 맥주 두 병, 오늘의 스페셜인 스테이크와 곁들일 완두콩과 당근, 디저트로 아이스크림을 주문합니다.


In [80]:
#========== summary 길이 이상치 찾기 ==========#
# dialogue도 기괴하게 긴데 summary가 과하게 긴지
# dialogue가 중간에 잘린 느낌인데 summary는 정상인 케이스
# → 이런 경우는 사실상 “input이 손상된 상태”라 학습에 노이즈가 됨

Q1_train = train_df["summary_len"].quantile(0.25)
Q3_train = train_df["summary_len"].quantile(0.75)
IQR_train = Q3_train - Q1_train
upper_train = Q3_train + 1.5 * IQR_train
long_summaries_train = train_df[train_df["summary_len"] >= upper_train]
lower_train = max(15, Q1_train - 1.5 * IQR_train)
short_summaries_train = train_df[train_df["summary_len"] <= lower_train]

print("✅ train_df")
print("📊 summary 길이 분포 (문장 수 기준)")
display(train_df["summary_n_sent"].value_counts().sort_index())

print("🔎 summary 문장 수 통계")
display(train_df["summary_n_sent"].describe())

print("📊 summary 길이 분포 (글자 수 기준)")
display(train_df["summary_len"].describe())

print("🔎 dialogue 대비 summary 길이 비율 분포")
display(train_df["len_ratio"].describe())

print(f"📌 summary_len IQR 기준 상단 이상치 threshold: {upper_train}")
print(f"⚠️ 과도하게 긴 summary 개수: {len(long_summaries_train)}")
display(long_summaries_train.sort_values("summary_len", ascending=False)[["fname", "dialogue_len", "summary_len", "summary"]].head())

print(f"📌 summary_len IQR 기준 하단 이상치 threshold: {lower_train}")
print(f"⚠️ 과도하게 짧은 summary 개수: {len(short_summaries_train)}")
display(short_summaries_train.sort_values("summary_len", ascending=True)[["fname", "dialogue_len", "summary_len", "summary"]].head())

#============================================================#

Q1_val = val_df["summary_len"].quantile(0.25)
Q3_val = val_df["summary_len"].quantile(0.75)
IQR_val = Q3_val - Q1_val
upper_val = Q3_val + 1.5 * IQR_val
long_summaries_val = val_df[val_df["summary_len"] >= upper_val]
lower_val = max(15, Q1_val - 1.5 * IQR_val)
short_summaries_val = val_df[val_df["summary_len"] <= lower_val]

print("✅ val_df")
print("📊 summary 길이 분포 (문장 수 기준)")
display(val_df["summary_n_sent"].value_counts().sort_index())

print("🔎 summary 문장 수 통계")
display(val_df["summary_n_sent"].describe())

print("📊 summary 길이 분포 (글자 수 기준)")
display(val_df["summary_len"].describe())

print("🔎 dialogue 대비 summary 길이 비율 분포")
display(val_df["len_ratio"].describe())

print(f"📌 summary_len IQR 기준 상단 이상치 threshold: {upper_val}")
print(f"⚠️ 과도하게 긴 summary 개수: {len(long_summaries_val)}")
display(long_summaries_val.sort_values("summary_len", ascending=False)[["fname", "dialogue_len", "summary_len", "summary"]].head())

print(f"📌 summary_len IQR 기준 하단 이상치 threshold: {lower_val}")
print(f"⚠️ 과도하게 짧은 summary 개수: {len(short_summaries_val)}")
display(short_summaries_val.sort_values("summary_len", ascending=True)[["fname", "dialogue_len", "summary_len", "summary"]].head())

✅ train_df
📊 summary 길이 분포 (문장 수 기준)


summary_n_sent
1    6036
2    4835
3    1212
4     269
5      66
6      20
7      13
8       5
9       1
Name: count, dtype: int64

🔎 summary 문장 수 통계


count    12457.000000
mean         1.686441
std          0.829337
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          9.000000
Name: summary_n_sent, dtype: float64

📊 summary 길이 분포 (글자 수 기준)


...

count    499.000000
mean       0.227963
std        0.092162
min        0.037179
25%        0.170308
50%        0.207317
75%        0.262274
max        0.923077
Name: len_ratio, dtype: float64

📌 summary_len IQR 기준 상단 이상치 threshold: 153.0
⚠️ 과도하게 긴 summary 개수: 16


,fname,dialogue_len,summary_len,summary
401,dev_401,1035,283,#Person1#은 #Person2#에게 더 이상 유효하지 않은 처방전을 재발급해달...
130,dev_130,890,216,#Person2#는 금목걸이를 도난당했다고 주장합니다. #Person1#은 몇 가지...
490,dev_491,746,208,#Person1#은 #Person2#에게 Bill Gates의 집에 대해 설명합니다...
289,dev_289,814,201,Steven은 임무 때문에 Veronica에게 거짓말을 했습니다. Veronica는...
410,dev_410,1052,201,#Person1#과 #Person2#는 금요일 밤에 어디로 갈지 논의합니다. #Pe...


📌 summary_len IQR 기준 하단 이상치 threshold: 15
⚠️ 과도하게 짧은 summary 개수: 0


,fname,dialogue_len,summary_len,summary


In [81]:
# dialog와 summary의 유사성 보기
print("✅ train_df")

print("📊 dialog와 summary의 중복률")
display(train_df[["uni_overlap", "bi_overlap"]].describe())

# overlap이 낮은(요약이 많이 추상적인) 샘플"
low_overlap = train_df.sort_values("uni_overlap").head(10)      # 유니그램 기준
print("🔎 유니그램 overlap 낮은 샘플")
display(low_overlap[["fname", "dialogue", "summary", "uni_overlap", "bi_overlap"]])

# overlap이 높은(거의 추출 요약 느낌) 샘플
high_overlap = train_df.sort_values("uni_overlap", ascending=False).head(10)
print("🔎 유니그램 overlap 높은 샘플")
display(high_overlap[["fname", "dialogue", "summary", "uni_overlap", "bi_overlap"]])


print("✅ val_df")

print("📊 dialog와 summary의 중복률")
display(val_df[["uni_overlap", "bi_overlap"]].describe())

# overlap이 낮은(요약이 많이 추상적인) 샘플"
low_overlap = val_df.sort_values("uni_overlap").head(10)      # 유니그램 기준
print("🔎 유니그램 overlap 낮은 샘플")
display(low_overlap[["fname", "dialogue", "summary", "uni_overlap", "bi_overlap"]])

# overlap이 높은(거의 추출 요약 느낌) 샘플
high_overlap = val_df.sort_values("uni_overlap", ascending=False).head(10)
print("🔎 유니그램 overlap 높은 샘플")
display(high_overlap[["fname", "dialogue", "summary", "uni_overlap", "bi_overlap"]])

✅ train_df
📊 dialog와 summary의 중복률


,uni_overlap,bi_overlap
count,12457.000000,12457.000000
mean,0.321208,0.065256
std,0.122986,0.080526
min,0.000000,0.000000
25%,0.235294,0.000000
50%,0.312500,0.047619
75%,0.400000,0.105263
max,0.866667,0.545455


🔎 유니그램 overlap 낮은 샘플


,fname,dialogue,summary,uni_overlap,bi_overlap
7710,train_7710,"#Person1#: 선생님, 사고를 분명히 보셨나요?\n#Person2#: 제가 우...",한 목격자가 사고 발생 상황을 자세히 설명했다.,0.0,0.0
8388,train_8388,"#Person1#: 안녕, Ed.\n#Person2#: 안녕, Jane. 또 만나서...","Jane과 Ed는 올해 같은 학교에서 10학년을 가르치게 되며, 두 사람 모두 그 ...",0.0,0.0
5759,train_5759,#Person1#: Bob!\n#Person2#: Anne! 정말 오랜만이야. 어떻...,오랜만에 만난 Bob과 Anne이 서로 인사하고 근황을 나눈다.,0.0,0.0
3801,train_3801,"#Person1#: 얘, Ann. 일어나. 이제 일어날 시간이야.\n#Person2...",Ann의 아빠는 Ann을 깨워 지각하지 않도록 합니다. Ann은 알람 소리를 듣지 ...,0.0,0.0
5878,train_5878,"#Person1#: 날씨가 참 화창하네요, 그렇죠?\n#Person2#: 맞아요. ...",David와 Jennifer는 처음 만나 서로 인사를 나누고 학업 관련 계획을 공유한다.,0.0,0.0
9437,train_9437,#Person1#: 여보세요. Kevin 좀 바꿔주실래요?\n#Person2#: 저...,"Joan이 Kevin에게 전화해 면접 일정을 알려주고, Kevin은 기쁘게 응답한다.",0.0,0.0
11462,train_11464,"#Person1#: 안녕하세요, 저는 Jack이에요. Christina 씨 맞으세요...",Jack과 Christina는 서로의 국적과 현재 거주지에 대해 이야기합니다.,0.0,0.0
5898,train_5898,"#Person1#: 안녕, 모니카.\n#Person2#: 안녕, 피터.\n#Pers...","피터가 모니카를 쇼에 초대하지만, 모니카는 바빠서 갈 수 없다.",0.0,0.0
11945,train_11948,"#Person1#: 안녕, Mikel. 무슨 일 있어? 화난 것 같은데.\n#Per...","Mikel은 체중이 늘어나서 고민하고 있는데, George가 함께 운동하자고 제안합...",0.0,0.0
958,train_958,"#Person1#: 안녕, 마이크. 무슨 일 있어?\n#Person2#: 안녕, 빌...","마이크는 새로운 차를 구입했고, 빌과 함께 드라이브를 가기로 했다.",0.0,0.0


🔎 유니그램 overlap 높은 샘플


...

,uni_overlap,bi_overlap
count,499.000000,499.000000
mean,0.308976,0.055006
std,0.114218,0.073106
min,0.000000,0.000000
25%,0.222222,0.000000
50%,0.307692,0.000000
75%,0.384615,0.095238
max,0.714286,0.384615


🔎 유니그램 overlap 낮은 샘플


,fname,dialogue,summary,uni_overlap,bi_overlap
196,dev_196,"#Person1#: 얼굴이 좀 안 좋아 보이네. 괜찮아, Tommy?\n#Perso...","Tommy가 사용 중이던 엄마의 휴대폰이 도난당했고, 엄마는 근처 경찰에게 즉시 신...",0.000000,0.0
215,dev_215,#Person1#: 새 영화 어땠어? 재밌었어?\n#Person2#: 그 영화 그냥...,"Carole은 Simon의 변명에 대해 Rebecca에게 이야기하고, Rebecca...",0.000000,0.0
136,dev_136,"#Person1#: 여보세요.\n#Person2#: 여보세요, 아빠?\n#Perso...",Mary가 아버지에게 가족의 안부를 물으며 통화한다.,0.000000,0.0
408,dev_408,#Person1#: 이린! 어젯밤에 데이트했다면서? 어땠어? 재미있는 이야기 다 해...,"아이린은 #Person1#에게 데이트 상대 데이비드의 외모, 직업 등을 이야기한다....",0.058824,0.0
110,dev_110,"#Person1#: Jenny, 얼굴이 안 좋아 보이는데 무슨 일 있어?\n#Per...",Jenny는 시험을 잘 보지 못해 학교를 그만두려는 생각을 하고 있다. Sally는...,0.066667,0.0
357,dev_357,"#Person1#: Adam, 미안해.\n#Person2#: Alice, 어디 있었...",Alice는 Adam과의 약속에 한 시간 이상 늦었다. 그녀는 늦은 이유를 설명하고...,0.083333,0.0
98,dev_98,"#Person1#: 서 이잭, 근처에 기독교 교회가 있어요?\n#Person2#: ...",#Person1#이 기독교를 연구하기 위해 Izek이 신부와의 만남을 주선해 주기로...,0.090909,0.0
211,dev_211,"#Person1#: 안녕, John. 잘 지냈어?\n#Person2#: 잘 지냈어,...","#Person1#는 John과 최근 근황에 대해 이야기를 나눈다. 또한, John이...",0.100000,0.0
77,dev_77,"#Person1#: 안녕 Kevin!\n#Person2#: 안녕 Laura, 오랜만...",Laura와 Kevin은 오랜만에 만나 서로의 최근 소식을 나눈다. Laura는 승...,0.100000,0.0
113,dev_113,"#Person1#: 안녕, 메리. 잘 지내고 있어?\n#Person2#: 글쎄, 어...","메리는 앤이 남자친구와의 약속으로 인해 만나지 못하자 다툼을 벌였고, #Person...",0.105263,0.0


🔎 유니그램 overlap 높은 샘플


,fname,dialogue,summary,uni_overlap,bi_overlap
491,dev_492,"#Person1#: 뭘 도와드릴까요, 손님?\n#Person2#: 오늘 아침에 뭐 ...","#Person2#는 토마토 주스, 크림 오브 위트 한 접시, 바삭한 베이컨과 프라이...",0.714286,0.384615
233,dev_233,"#Person1#: 수잔, 너 고대 중국에서 가장 못생긴 네 명의 여인에 대해 들어...",#Person1#은 수잔에게 고대 중국에서 덕이 훌륭했던 네 명의 가장 못생긴 여인...,0.642857,0.230769
32,dev_32,"#Person1#: 저기요, 이 버스가 중국 국가 도서관으로 가나요?\n#Perso...",#Person1#이 #Person2#에게 도착하면 정거장에 알려달라고 부탁합니다.,0.625000,0.000000
102,dev_102,"#Person1#: 안녕하세요, 노트북을 가지고 왔는데 제 방에서 인터넷을 어떻게 ...",#Person2#는 #Person1#에게 호텔 12층에 있는 인터넷 라운지를 이용할...,0.615385,0.333333
48,dev_48,#Person1#: 도와드릴까요?\n#Person2#: 다음 주 일요일 전까지 캐나...,고객은 다음 주 일요일 전까지 캘거리행 이코노미 클래스 편도 비행기 티켓 3장을 예...,0.611111,0.294118
256,dev_256,"#Person1#: 안녕하세요, 여기가 Bread and Cheese Deli인가요...",#Person1#님이 Bread and Cheese Deli에서 샌드위치와 음료를 ...,0.600000,0.222222
168,dev_168,#Person1#: 이번 크리스마스에는 우리 가족을 위해 특별한 걸 해보자.\n#P...,#Person1#과 #Person2#는 이번 크리스마스에 가족을 위해 특별한 일을 ...,0.588235,0.250000
269,dev_269,"#Person1#: 안녕 존, 다음 주에 있을 회의에 우리 고객들을 모두 초대해 줘...","#Person1#는 #Person2#에게 다음 주 회의에 고객들을 초대하고, 회의 ...",0.578947,0.111111
66,dev_66,"#Person1#: 저기요, 이 자리 비었나요?\n#Person2#: 비어 있어요....","#Person1#이 자리 공유를 요청하고, #Person2#이 동의합니다.",0.571429,0.142857
82,dev_82,#Person1#: 일자리를 찾고 싶어요.\n#Person2#: 여기 여러 파트타임...,#Person1#은 컴퓨터 센터의 일자리에 지원하려고 합니다.,0.571429,0.333333


In [82]:
# dialogue와 summary의 중복률 눈으로 보기

p=491
print("✅ dialogue")
print(train_df.loc[p]["dialogue"])

print("\n✅ summary")
print(train_df.loc[p]["summary"])

# overlap이 높은 데이터셋이면
# → 전처리에서 “원문 의미 보존 + 불필요 토큰만 제거” 방향이 맞고
# → 복사 성향 강한 요약기가 잘 버팀

# overlap이 낮으면
# → 더 고급 생성이 필요하고, EDA를 통해 핵심 발화만 남기는 전처리가 중요해져.

✅ dialogue
#Person1#: 안녕하세요, 여러분. 훈련 워크숍에 참여해 주셔서 감사합니다. 오늘 멋진 연사들이 기다리고 있습니다. 첫 번째로 모실 분은 국제 경제 개발 분야에서 유명한 하버드의 Dr. James Smith입니다. Dr. Smith는 20년 넘게 경제 연구에 참여하고 있으며, 1995년부터 하버드에서 가르치고 있습니다. 그럼, Dr. James Smith를 환영합니다.
#Person2#: 감사합니다, Mr. Jackson. 안녕하세요, 여러분. 제 강연 주제는 현대 세계의 국제 관계입니다. 중동의 현재 상황과 그것이 세계 경제에 미치는 영향을 간단히 말씀드리겠습니다. 또한, 겉으로는 관련 없어 보이는 국가들의 경제가 어떻게 서로 얽혀 있는지 설명하겠습니다. 제 강연은 세 부분으로 이루어집니다. 첫 번째는 국제 관계의 개요, 두 번째는 현재 정치 상황에 대한 논의, 마지막으로 미래를 위한 트렌드입니다. 발표는 약 두 시간 걸리고, 중간에 20분 휴식이 있습니다. 점심은 12시에 제공합니다.

✅ summary
Mr. Jackson이 국제 경제 개발 전문가 Dr. James Smith를 소개합니다. Dr. Smith는 현대 세계의 국제 관계에 대해 강연할 예정이며, 강연의 주제와 일정을 안내합니다.


In [83]:
print("📊 요약과 가장 유사한 dialogue 줄의 위치")   # 평균 3~4번째 줄
print("✅ train_df")
display(train_df["summary_main_turn"].describe())
print("✅ val_df")
display(val_df["summary_main_turn"].describe())

📊 요약과 가장 유사한 dialogue 줄의 위치
✅ train_df


count    12457.000000
mean         3.837441
std          3.659871
min          0.000000
25%          1.000000
50%          3.000000
75%          6.000000
max         46.000000
Name: summary_main_turn, dtype: float64

✅ val_df


count    499.000000
mean       3.645291
std        3.489576
min        0.000000
25%        1.000000
50%        3.000000
75%        6.000000
max       21.000000
Name: summary_main_turn, dtype: float64

In [84]:
#========== 간단한 토큰화 함수 ==========#
def tokenize(t):
    t = re.sub(r"[^가-힣A-Za-z0-9 ]", " ", t)
    return [w for w in t.split() if len(w) > 1]     # 2개 합칠 수 있나?

# 실행
all_summary_tokens_train = []
all_summary_tokens_val = []

for s in train_df["summary"]:
    all_summary_tokens_train.extend(simple_tokenize(s))
summary_freq_train = Counter(all_summary_tokens_train)

for t in val_df["summary"]:
    all_summary_tokens_val.extend(simple_tokenize(t))
summary_freq_val = Counter(all_summary_tokens_val)


#========== 의미있는 토큰 확인하는 함수 ==========#
def is_meaningful_token(tok: str) -> bool:
    if len(tok) <= 1:   # 1글자 조사/부사 등 너무 짧은 건 제외
        return False
    if tok.isdigit():   # 숫자만 있는 토큰 제외
        return False
    return True

meaningful_freq_train = Counter([u for u in all_summary_tokens_train if is_meaningful_token(u)])
meaningful_freq_val = Counter([u for u in all_summary_tokens_val if is_meaningful_token(u)])

#========== Summary에 남는 / 삭제되는 키워드 확인하는 함수 ==========#
kept_counter_train = Counter(sum(train_df["keywords_kept"], []))
kept_counter_val = Counter(sum(val_df["keywords_kept"], []))
dropped_counter_train = Counter(sum(train_df["keywords_dropped"], []))
dropped_counter_val = Counter(sum(val_df["keywords_dropped"], []))

In [85]:
print("✅ train_df")

print("📊 summary 토큰 상위 50개")
for token, cnt in summary_freq_train.most_common(10):
    print(f"{token:15} {cnt}")

print("\n📊 의미 있는 summary 토큰 상위 50개")
for token, cnt in meaningful_freq_train.most_common(10):
    print(f"{token:15} {cnt}")

print("\n🔎 요약에 유지되는 핵심 단어")
for i in kept_counter_train.most_common(10):
    print(i)

print("\n🔎 요약에서 제거되는 단어")
for j in dropped_counter_train.most_common(10):
    print(j)


print("\n✅ val_df")

print("📊 summary 토큰 상위 50개")
for token, cnt in summary_freq_val.most_common(10):
    print(f"{token:15} {cnt}")

print("\n📊 의미 있는 summary 토큰 상위 50개")
for token, cnt in meaningful_freq_val.most_common(10):
    print(f"{token:15} {cnt}")

print("\n🔎 요약에 유지되는 핵심 단어")
for i in kept_counter_val.most_common(10):
    print(i)

print("\n🔎 요약에서 제거되는 단어")
for j in dropped_counter_val.most_common(10):
    print(j)

✅ train_df
📊 summary 토큰 상위 50개
person1         13415
person2         12553
는               7971
은               6691
에게              4334
대해              2257
이               1878
의               1661
가               1500
과               1314

📊 의미 있는 summary 토큰 상위 50개
person1         13415
person2         12553
에게              4334
대해              2257
합니다             1136
위해              1130
설명합니다           1074
것을              838
있습니다            826
말합니다            722

🔎 요약에 유지되는 핵심 단어
('person1', 10095)
('person2', 8780)
('수', 732)
('이', 493)
('더', 490)
('대해', 425)
('mr', 360)
('그', 256)
('두', 254)
('새', 235)

🔎 요약에서 제거되는 단어
('정말', 4901)
('수', 4760)
('것', 4318)
('좀', 4282)
('person2', 3677)
('안', 3599)
('있어', 3468)
('거야', 3292)
('그럼', 2917)
('어떻게', 2836)

✅ val_df
📊 summary 토큰 상위 50개
person1         552
person2         510
는               302
은               267
에게              174
대해              106
이               93
과               73
가               68
의               64

📊

In [86]:
# ✅ train_df
# 📊 summary 토큰 상위 50개
# person1         13415
# person2         12469
# 는               7971
# 은               6691
# 에게              4334
# 대해              2257
# 이               1878
# 의               1661
# 가               1500
# 과               1314

# 📊 의미 있는 summary 토큰 상위 50개
# person1         13415
# person2         12469
# 에게              4334
# 대해              2257
# 합니다             1136
# 위해              1130
# 설명합니다           1074
# 것을              838
# 있습니다            826
# 말합니다            722

# 🔎 요약에 유지되는 핵심 단어
# ('person1', 10095)
# ('person2', 8768)
# ('수', 732)
# ('이', 492)
# ('더', 490)
# ('대해', 425)
# ('mr', 360)
# ('그', 256)
# ('두', 254)
# ('새', 235)

# 🔎 요약에서 제거되는 단어
# ('정말', 4901)
# ('네', 4834)
# ('수', 4760)
# ('것', 4318)
# ('좀', 4282)
# ('person2', 3689)
# ('안', 3599)
# ('있어', 3468)
# ('거야', 3292)
# ('아', 3266)


# ✅ val_df
# 📊 summary 토큰 상위 50개
# person1         552
# person2         509
# 는               302
# 은               267
# 에게              174
# 대해              106
# 이               93
# 과               73
# 가               68
# 의               64

# 📊 의미 있는 summary 토큰 상위 50개
# person1         552
# person2         509
# 에게              174
# 대해              106
# 위해              39
# 합니다             39
# 것을              34
# 설명합니다           33
# 있는              31
# 이야기합니다          28

# 🔎 요약에 유지되는 핵심 단어
# ('person1', 415)
# ('person2', 370)
# ('이', 27)
# ('더', 25)
# ('수', 22)
# ('대해', 18)
# ('있는', 15)
# ('두', 12)
# ('새', 11)
# ('다시', 10)

# 🔎 요약에서 제거되는 단어
# ('수', 212)
# ('정말', 209)
# ('네', 185)
# ('것', 177)
# ('좀', 176)
# ('있어', 145)
# ('안', 138)
# ('거야', 130)
# ('person2', 129)
# ('아', 121)

In [87]:
# 너무 당연한 stopword스러운 애들(예: “는”, “을”, “에”, “의”)이 많으면
# → 나중에 불용어 리스트를 만들어서 제거해도 됨

# 대신에 예약, 방문, 호텔, 차량, 대여, 준비, 회의, 시험, 병원 같은 키워드들을 보면
# → 이게 사실상 task domain이야.

In [88]:
print('✅ train_df')

print("🔎 summary 끝 3글자 상위 20개 | 총 :", train_df["summary_ending3"].value_counts().shape[0])
display(train_df["summary_ending3"].value_counts().head(10))

print("🔎 summary 끝 2글자 상위 20개 | 총 :", train_df["summary_ending2"].value_counts().shape[0])
display(train_df["summary_ending2"].value_counts().head(10))

print('✅ val_df')

print("🔎 summary 끝 3글자 상위 20개 | 총 :", val_df["summary_ending3"].value_counts().shape[0])
display(val_df["summary_ending3"].value_counts().head(10))

print("🔎 summary 끝 2글자 상위 20개 | 총 :", val_df["summary_ending2"].value_counts().shape[0])
display(val_df["summary_ending2"].value_counts().head(10))

✅ train_df
🔎 summary 끝 3글자 상위 20개 | 총 : 84


summary_ending3
니다.    9189
한다.    2076
했다.     195
있다.     191
이다.     182
는다.      87
준다.      84
었다.      33
낀다.      28
된다.      27
Name: count, dtype: int64

🔎 summary 끝 2글자 상위 20개 | 총 : 16


summary_ending2
다.    12428
대화        6
함.        5
화.        4
황.        2
논의        2
인.        1
기.        1
해결        1
이견        1
Name: count, dtype: int64

✅ val_df
🔎 summary 끝 3글자 상위 20개 | 총 : 27


summary_ending3
니다.    356
한다.     92
는다.      7
했다.      6
있다.      6
이다.      5
준다.      4
린다.      2
눈다.      2
낀다.      2
Name: count, dtype: int64

🔎 summary 끝 2글자 상위 20개 | 총 : 2


summary_ending2
다.    498
기.      1
Name: count, dtype: int64

### Topic

In [89]:
print('✅ train_df')

print("🔎 ''(빈 문자열) topic 개수 :", (train_df["topic_stripped"] == "").sum())

print("\n🔎 topic 길이 분포")
display(train_df["topic_len"].describe())

# 비정상적으로 긴 topic 확인 (예: 상위 1% 이상)
t_q99_train = train_df["topic_len"].quantile(0.99)
weird_topic_train = train_df[train_df["topic_len"] >= t_q99_train]

print(f"⚠️ topic_len 상위 1% 이상 (threshold={t_q99_train}) 샘플 :", weird_topic_train[["fname", "topic", "topic_len"]].shape[0])
display(weird_topic_train.sort_values("topic_len", ascending=False)[["fname", "topic", "topic_len"]].head(10))


print('✅ val_df')

print("🔎 ''(빈 문자열) topic 개수 :", (val_df["topic_stripped"] == "").sum())

print("\n🔎 topic 길이 분포")
display(val_df["topic_len"].describe())

# 비정상적으로 긴 topic 확인 (예: 상위 1% 이상)
t_q99_val = val_df["topic_len"].quantile(0.99)
weird_topic_val = val_df[val_df["topic_len"] >= t_q99_val]

print(f"⚠️ topic_len 상위 1% 이상 (threshold={t_q99_val}) 샘플 :", weird_topic_val[["fname", "topic", "topic_len"]].shape[0])
display(weird_topic_val.sort_values("topic_len", ascending=False)[["fname", "topic", "topic_len"]].head(10))

✅ train_df
🔎 ''(빈 문자열) topic 개수 : 0

🔎 topic 길이 분포


count    12457.000000
mean         7.497471
std          2.884445
min          1.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         30.000000
Name: topic_len, dtype: float64

⚠️ topic_len 상위 1% 이상 (threshold=16.0) 샘플 : 162


,fname,topic,topic_len
9096,train_9096,생일 파티와 'Happy Birthday' 노래의 기원,30
1296,train_1296,해외에서 일하는 런던 출신 디자이너의 경험,23
10039,train_10039,Student Experience 웹사이트,23
9800,train_9800,'The Power of One' 책 읽기,23
6449,train_6449,비행기 좌석 조절 및 귀 압력 문제 해결,22
7567,train_7567,Denys Hawtin에 대한 정보 요청,22
6236,train_6236,비행기 내 식사 주문 및 자리 변경 요청,22
6043,train_6043,여자친구의 차 빌리기 및 어머니와의 저녁,22
12387,train_12390,뉴욕에서 애틀랜틱 시티로 가는 4일 여행,22
6951,train_6951,Fitzgerald 감독의 영화와 성격,21


✅ val_df
🔎 ''(빈 문자열) topic 개수 : 0

🔎 topic 길이 분포


count    499.000000
mean       7.683367
std        2.855268
min        1.000000
25%        5.000000
50%        7.000000
75%        9.000000
max       18.000000
Name: topic_len, dtype: float64

⚠️ topic_len 상위 1% 이상 (threshold=16.0) 샘플 : 7


,fname,topic,topic_len
453,dev_453,영화 등급에 대한 관람 태도 변화,18
132,dev_132,Green Peace 활동 소개,17
481,dev_482,헬스클럽 규정에 따른 복장 제한,17
207,dev_207,처방전 수령 및 약 복용 상담,16
284,dev_284,이혼과 아이들에게 미치는 영향,16
340,dev_340,새 노트북과 카페 무선 인터넷,16
438,dev_438,Rose의 임신과 가족의 기대,16


In [90]:
# 비슷한데 다르게 저장되어 있는 topic이 있는지 확인
topic_counts_train = train_df["topic_stripped"].value_counts()
topic_counts_val = val_df["topic_stripped"].value_counts()

print("📊 topic 상위 30개")
print("✅ train_df | 총 :", topic_counts_train.shape[0])
display(topic_counts_train.head(30))

print("\n✅ val_df | 총 :", topic_counts_val.shape[0])
display(topic_counts_val.head(30))

# 있음! ex. 호텔 예약 vs 호텔 객실 예약 등
# 근데 이거를 어떻게 다 찾고, 어떻게 취합해야 할지...

📊 topic 상위 30개
✅ train_df | 총 : 9235


topic_stripped
음식 주문        130
취업 면접        109
길 안내          66
호텔 체크인        40
아파트 임대        30
일상 대화         29
쇼핑            27
주말 계획         26
면접            25
호텔 예약         25
여행 계획         23
항공권 예약        22
영화 감상         21
작별 인사         21
휴가 계획         21
가격 협상         20
택시 이용         20
영화 관람 계획      18
의료 상담         16
파티 초대         16
길 묻기          15
호텔 객실 예약      15
저녁 초대         15
환전            15
호텔 체크아웃       15
호텔 방 예약       14
전화 메시지 전달     14
체중 감량         14
소포 발송         14
전화 통화         14
Name: count, dtype: int64


✅ val_df | 총 : 475


topic_stripped
호텔 방 예약      5
길 안내         4
취업 면접        4
음식 주문        4
신발 구매        2
월급 인상        2
비행기 예약       2
애완동물 선택      2
계산서 오류       2
버스 이용        2
아파트 임대 협상    2
약속 지각        2
일상 대화        2
가격 협상        2
꿈의 집         2
여행 서비스 불만    1
생일 선물 불만     1
이탈리아 여행      1
아파트 임대       1
책 빌리기        1
시험 주문 협상     1
집들이 초대       1
고혈압 관리       1
침실 세트 구매     1
국제 송금        1
스테레오 보증      1
운동과 기분       1
향후 일정 조율     1
교통법규 위반      1
룸서비스 요청      1
Name: count, dtype: int64

In [91]:
# 토픽 별 빈도 구간으로 나누어보기
def bucket_count(c):
    if c >= 300:
        return "300+"
    elif c >= 200:
        return "200~299"
    elif c >= 100:
        return "100~199"
    elif c >= 50:
        return "50~99"
    elif c >= 20:
        return "20~49"
    else:
        return "<20"
    
topic_bucket_train = topic_counts_train.to_frame(name="count")
topic_bucket_train["bucket"] = topic_bucket_train["count"].apply(bucket_count)
topic_bucket_val = topic_counts_val.to_frame(name="count")
topic_bucket_val["bucket"] = topic_bucket_val["count"].apply(bucket_count)

top_10_ratio_train = topic_counts_train.head(10).sum() / len(train_df)
rare_topics_train = topic_counts_train[topic_counts_train < 5]
rare_sample_train = train_df[train_df["topic_stripped"].isin(rare_topics_train.index)].sample(min(20, len(rare_topics_train)), random_state=42)
top_10_ratio_val = topic_counts_val.head(10).sum() / len(val_df)
rare_topics_val = topic_counts_val[topic_counts_val < 5]
rare_sample_val = val_df[val_df["topic_stripped"].isin(rare_topics_val.index)].sample(min(20, len(rare_topics_val)), random_state=42)

In [92]:
print("✅ train_df")
print("📊 topic 빈도 통계")
display(topic_counts_train.describe())

print("📊 topic 빈도 구간별 개수")
display(topic_bucket_train["bucket"].value_counts())

print("\n🔎 topic 빈도 상위 10개 | 총 :", topic_counts_train.shape[0])
print(f"    - 상위 10개 topic이 전체 데이터에서 차지하는 비율 : {top_10_ratio_train:.3f}")         # 많지는 않은 듯?
display(topic_counts_train.head(10))

print("⚠️ 빈도 5 미만 rare topic 개수 :", len(rare_topics_train))
display(rare_sample_train[["fname", "topic_stripped", "summary"]].head(10))

print("✅ val_df")
print("📊 topic 빈도 통계")
display(topic_counts_val.describe())

print("📊 topic 빈도 구간별 개수")
display(topic_bucket_val["bucket"].value_counts())

print("\n🔎 topic 빈도 상위 10개 | 총 :", topic_counts_val.shape[0])
print(f"    - 상위 10개 topic이 전체 데이터에서 차지하는 비율 : {top_10_ratio_val:.3f}")         # 많지는 않은 듯?
display(topic_counts_val.head(10))

print("⚠️ 빈도 5 미만 rare topic 개수 :", len(rare_topics_val))
display(rare_sample_val[["fname", "topic_stripped", "summary"]].head(10))

# 진짜로 “매우 특수한 상황이라 rare인 topic”인지
# 아니면 오타 / 표현 차이로 쪼개진 건지 감이 와.

✅ train_df
📊 topic 빈도 통계


count    9235.000000
mean        1.348890
std         2.365001
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max       130.000000
Name: count, dtype: float64

📊 topic 빈도 구간별 개수


bucket
<20        9218
20~49        14
100~199       2
50~99         1
Name: count, dtype: int64


🔎 topic 빈도 상위 10개 | 총 : 9235
    - 상위 10개 topic이 전체 데이터에서 차지하는 비율 : 0.041


...

bucket
<20    475
Name: count, dtype: int64


🔎 topic 빈도 상위 10개 | 총 : 475
    - 상위 10개 topic이 전체 데이터에서 차지하는 비율 : 0.058


topic_stripped
호텔 방 예약    5
길 안내       4
취업 면접      4
음식 주문      4
신발 구매      2
월급 인상      2
비행기 예약     2
애완동물 선택    2
계산서 오류     2
버스 이용      2
Name: count, dtype: int64

⚠️ 빈도 5 미만 rare topic 개수 : 474


,fname,topic_stripped,summary
460,dev_460,문제 있는 사람과의 거리두기,"#Person1#은 #Person2#에게 어떤 남자와 거리를 두라고 권유하고, #P..."
74,dev_74,주차 공간 문제,#Person1#은 #Person2#에게 자신의 주차 공간에 주차한 차를 이동해달라...
313,dev_313,만족스러운 헤어 스타일,"#Person1#은 헤어 스타일에 만족하며, 55달러를 지불했습니다."
177,dev_177,영화 티켓 예약,#Person2#는 #Person1#이 토요일에 Dark and Stormy Nig...
335,dev_335,향후 일정 조율,#Person2#가 #Person1#에게 향후 이틀간의 일정과 계획을 설명한다.
207,dev_207,처방전 수령 및 약 복용 상담,#Person1#은 처방전을 받기 위해 #Person2#를 방문합니다. 또한 #Pe...
157,dev_157,옷 쇼핑과 교환,#Person2#는 #Person1#이 매일 옷을 고민하는 것에 대해 체념하고 있습...
55,dev_55,소음 불만,Mr. Black은 아이들의 소음에 대해 Mr. Brown에게 불평합니다.
323,dev_323,길 안내,"#Person2#는 #Person1#에게 슈퍼마켓, 세탁소 및 이발소의 위치를 알려..."
9,dev_9,꿈의 집,#Person1#과 #Person2#가 Bill Gate의 집에 대해 이야기합니다....


In [93]:
# <20 구간이 엄청 많다 → rare topic이 많다는 뜻
# 이런 rare topic 구간은:
# train / dev split 시 stratified split by topic 고려
# 또는 데이터 증강, oversampling 후보

In [94]:
# topic별 summary key token 보기
topic_token_stats_train = {}
topic_token_stats_val = {}

for t in train_df["topic"].unique():
    sub = train_df[train_df["topic"] == t]
    tokens = []
    for s in sub["summary"]:
        tokens.extend(simple_tokenize(s))
    freq = Counter([tok for tok in tokens if is_meaningful_token(tok)])
    topic_token_stats_train[t] = freq

for t in val_df["topic"].unique():
    sub = val_df[val_df["topic"] == t]
    tokens = []
    for u in sub["summary"]:
        tokens.extend(simple_tokenize(u))
    freq = Counter([tok for tok in tokens if is_meaningful_token(tok)])
    topic_token_stats_val[t] = freq

# topic 별 상위 토큰 보기
t = "취업 면접"                 # 다른 topic도 확인해보기
print(f"🔎 Topic = {t} 의 summary 핵심 토큰")
print("✅ train_df")
for tok, cnt in topic_token_stats_train[t].most_common(10):
    print(f"{tok:15} {cnt}")
print("\n✅ val_df")
for tok, cnt in topic_token_stats_val[t].most_common(10):
    print(f"{tok:15} {cnt}")

🔎 Topic = 취업 면접 의 summary 핵심 토큰
✅ train_df
person2         113
person1         112
대해              48
에게              41
mr              31
설명합니다           29
자신의             18
면접을             14
이유를             13
있다고             12

✅ val_df
person2         5
person1         5
대해              4
자신의             2
에게              1
미래              1
계획과             1
이전              1
업무              1
경험에             1


In [95]:
# Topic과 Summary 관계

print("✅ train_df")

print("📊 topic 문자열이 summary에 '직접 등장'하는 비율")
display(train_df["topic_in_summary"].value_counts(normalize=True).rename(lambda x: "포함" if x else "미포함").mul(100).round(2))

print("🔎 topic별 topic-in-summary 비율 (상위 10개)")
topic_stats = (train_df.groupby("topic")["topic_in_summary"].mean().sort_values(ascending=False).head(10).mul(100).round(2))
display(topic_stats)

print("📊 topic 단어가 summary에 '반영'되는 비율")
display(train_df["topic_coverage"].describe())

# coverage가 낮은 샘플 몇 개 확인
low_cov = train_df[train_df["topic_coverage"] < 0.5].sample(3, random_state=42)

for i, row in low_cov.iterrows():
    print("🔎 topic 단어가 summary에 반영되지 않은 케이스")
    print(f"[fname]             {row['fname']}")
    print(f"[topic]             {row['topic']}")
    print(f"[summary]           {row['summary']}")
    print(f"[topic_coverage]    {row['topic_coverage']:.2f}\n")


print("✅ val_df")

print("📊 topic 문자열이 summary에 '직접 등장'하는 비율")
display(val_df["topic_in_summary"].value_counts(normalize=True).rename(lambda x: "포함" if x else "미포함").mul(100).round(2))

print("🔎 topic별 topic-in-summary 비율 (상위 10개)")
topic_stats = (val_df.groupby("topic")["topic_in_summary"].mean().sort_values(ascending=False).head(10).mul(100).round(2))
display(topic_stats)

print("📊 topic 단어가 summary에 '반영'되는 비율")
display(val_df["topic_coverage"].describe())

# coverage가 낮은 샘플 몇 개 확인
low_cov = val_df[val_df["topic_coverage"] < 0.5].sample(3, random_state=42)

for i, row in low_cov.iterrows():
    print("🔎 topic 단어가 summary에 반영되지 않은 케이스")
    print(f"[fname]             {row['fname']}")
    print(f"[topic]             {row['topic']}")
    print(f"[summary]           {row['summary']}")
    print(f"[topic_coverage]    {row['topic_coverage']:.2f}\n")

✅ train_df
📊 topic 문자열이 summary에 '직접 등장'하는 비율


topic_in_summary
미포함    90.28
포함      9.72
Name: proportion, dtype: float64

🔎 topic별 topic-in-summary 비율 (상위 10개)


topic
사진 현상     100.0
출산        100.0
급여 합의     100.0
불길한 숫자    100.0
모닝콜       100.0
결혼        100.0
불꽃놀이      100.0
탱고        100.0
결혼 관습     100.0
잘못된 번호    100.0
Name: topic_in_summary, dtype: float64

📊 topic 단어가 summary에 '반영'되는 비율


...

🔎 topic별 topic-in-summary 비율 (상위 10개)


topic
수업 등록           100.0
예금 계좌 개설        100.0
결혼식             100.0
모자 퍼레이드         100.0
모래 폭풍           100.0
저녁 식사           100.0
매운 땅콩 닭고기 요리    100.0
작별 인사           100.0
과제 문제           100.0
광고              100.0
Name: topic_in_summary, dtype: float64

📊 topic 단어가 summary에 '반영'되는 비율


count    499.000000
mean       0.580995
std        0.336126
min        0.000000
25%        0.333333
50%        0.500000
75%        1.000000
max        1.000000
Name: topic_coverage, dtype: float64

🔎 topic 단어가 summary에 반영되지 않은 케이스
[fname]             dev_218
[topic]             집안일
[summary]           #Person1#이 엄마를 도와 요리를 준비하고 있습니다. 엄마는 #Person1#에게 아빠와 다니엘이 무엇을 하고 있는지 묻습니다.
[topic_coverage]    0.00

🔎 topic 단어가 summary에 반영되지 않은 케이스
[fname]             dev_163
[topic]             직장 내 스트레스
[summary]           에밀리는 #Person1#에게 매니저가 실수를 자신에게 뒤집어씌운 탓에 나쁜 하루를 보냈다고 이야기한다. 에밀리는 매니저가 모두에게 특히 자신에게 불공평하게 대하지만, 승진할 때까지 참아보겠다고 한다. #Person1#은 에밀리가 현명하다고 생각한다.
[topic_coverage]    0.00

🔎 topic 단어가 summary에 반영되지 않은 케이스
[fname]             dev_62
[topic]             잘못 내린 정류장
[summary]           #Person1#과 #Person2#는 #Person2#의 판단 실수로 인해 버스에서 너무 일찍 내렸습니다.
[topic_coverage]    0.00



In [96]:
# print("\n🔎 topic별 summary 예시 (topic마다 최대 3개)")
# for t in train_df["topic"].dropna().unique()[:5]:
#     sub = train_df[train_df["topic"] == t]
#     n = min(3, len(sub))          # 데이터 개수보다 많이 뽑지 않기
#     if n == 0:
#         continue
#     print(f"\n✅ Topic: {t} (n={len(sub)})")
#     sample = sub.sample(n=n, random_state=42)
#     display(sample[["fname", "summary"]])

In [97]:
# test_df에는 "topic"이 따로 없는데 이것도 지정해주면 요약을 더 정확히 하는데 도움이 될까?

# topic classifier + topic-consistent summarization (멀티 인풋 방식)

# ⭐ 마지막 결론
# test.csv에 topic이 없어도 topic을 예측해서 넣는 구조는 오히려 성능 최강 전략이다.
# ✔ 요약의 품질을 강하게 안정시키고
# ✔ summary의 형태를 topic별 평균 스타일에 가깝게 유지시킴
# ✔ LB 점수 상승 확률 매우 높음